# 599 Capstone Project

The notebook is for text data preprocessing.

## Globally import libraries and set display parameters

Libraries needed mostly pertain to dataframe manipulation for data preprocessing.

In [1]:
from collections import defaultdict, Counter
import datetime as dt
import emoji
from icecream import ic
from IPython.display import display_html 
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import random
import re
import regex as rex
import shutil
from string import punctuation
import time
from tqdm import tqdm

import nltk
from nltk.corpus import stopwords

import spacy

from sklearn.feature_extraction.text import TfidfTransformer, \
CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, Pipeline

import textacy.preprocessing as tprep
from textacy.extract import keyword_in_context

Set global parameters.

In [2]:
global_start_time = time.perf_counter()

random.seed(1699)

# Set pandas global options
pd.options.display.max_rows = 23
pd.options.display.precision = 4
np.set_printoptions(suppress=True,
                    precision=4)

%matplotlib inline

# Set tqdm package progress bar
tqdm.pandas(ncols=50)

dply_rng_end01 = 0
dply_rng_end02 = 2

to_csv_flag = False

## Upload data from CSV

Establish working directories for saving dataframes as CSV files.

In [3]:
'''Dir nav citation:
https://softhints.com/python-change-directory-parent/
'''
curr_dir = os.path.abspath(os.curdir)
ic(curr_dir)
os.chdir("..")
up1_dir = os.path.abspath(os.curdir)
ic(up1_dir)
ic()

ic| curr_dir: 'C:\\Users\\acarr\\Documents\\GitHub\\599_team_project\\deliverables'
ic| up1_dir: 'C:\\Users\\acarr\\Documents\\GitHub\\599_team_project'
ic| 652109981.py:9 in <module> at 09:55:15.641


Get current date/time to append to file name string.

In [4]:
today = dt.datetime.today()
today= str(today)
today = today.replace(':', '-').replace('.', '').replace(' ', '_')
ic(today)
ic(type(today))
ic()

ic| today: '2023-07-23_09-55-15672808'
ic| type(today): <class 'str'>
ic| 3990198718.py:6 in <module> at 09:55:15.724


Establish full file name path.

In [5]:
# change `data_location` to the location of the folder on your machine.
data_location = 'data'
ref_docs_location = 'ref_docs'

file_in_name01 = 'data_scraped_amc_2023-07-10_13-41-28670390.csv'

file_in_path01 = os.path.join(up1_dir, data_location, file_in_name01)

print(f'CSV file in 1 path: {file_in_path01}')

CSV file in 1 path: C:\Users\acarr\Documents\GitHub\599_team_project\data\data_scraped_amc_2023-07-10_13-41-28670390.csv


### Review dataframe

Read in data from CSV, check resulting dataframe shape, and display first several records.

In [6]:
slct_tbl_full_df01 = pd.read_csv(file_in_path01)
print(f'Dataframe shape: {slct_tbl_full_df01.shape}')
display(slct_tbl_full_df01.head())

Dataframe shape: (36420, 8)


,text_id,source_name,author,title,url,publish_date,article_text,content
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...


In [7]:
slct_tbl_full_df01a = \
slct_tbl_full_df01.loc[slct_tbl_full_df01['source_name'] == 'msnNOW']

print(f'Dataframe shape: {slct_tbl_full_df01a.shape}')
display(slct_tbl_full_df01a)

Dataframe shape: (13, 8)


,text_id,source_name,author,title,url,publish_date,article_text,content
7011,8492,msnNOW,NaN,Ancient Maya city discovered in Mexican jungle...,https://www.msn.com/en-us/news/world/ancient-m...,2023-06-21T13:25:37Z,Share to FacebookShare to TwitterShare to Link...,MEXICO CITY (Reuters) - A previously unknown a...
7590,9080,msnNOW,msn,Rome to open ancient square where Julius Caesa...,https://www.msn.com/,2023-06-20T13:00:05Z,"A ""Sold"" sign outside a home in the Toll Broth...",Rome to open ancient square where Julius Caesa...
24893,27083,msnNOW,NaN,Quebec expands assisted dying...,https://www.msn.com/en-us/health/other/canadas...,2023-06-15T02:40:40Z,\n\n\n\n\nMore On:\n\n\t\t\t\t\trebel wilson\t...,"Colette Julien, like thousands of sick Canadia..."
25002,27195,msnNOW,NaN,Norway to Mandate 40% Quota for Women on Compa...,https://www.msn.com/en-us/money/markets/norway...,2023-06-19T15:52:38Z,\n\n\n\n\nMore On:\n\n\t\t\t\t\tgrammys\t\t\t\...,"OSLO, June 19 (Reuters) - Large and mid-size p..."
25026,27220,msnNOW,Afp 5 Hrs Ago,Applying vaginal fluid to C-section babies boo...,https://www.msn.com/en-us/health/other/applyin...,2023-06-15T20:31:46Z,\n\n\n\n\nMore On:\n\n\t\t\t\t\tthe weeknd\t\t...,Babies born by cesarean section don't acquire ...
26462,28680,msnNOW,msn,"GOOGLE, one of AI's biggest backers, warns own...",https://www.msn.com/en-us/news/technology/goog...,2023-06-15T13:00:04Z,\n\n\n\n\nMore On:\n\n\t\t\t\t\tlife\t\t\t\t\n...,"Google, one of AI’s biggest backers, warns own..."
27385,29618,msnNOW,Matthew Brown,Youth environmentalists bring Montana climate ...,https://www.msn.com/en-us/health/health-news/y...,2023-06-11T04:34:16Z,In this articleBOWLFollow your favorite stocks...,"HELENA, Mont. (AP) Whether a constitutional ri..."
28846,31099,msnNOW,Thalia Beaty,Transgender adults in Florida `blindsided' tha...,http://www.msn.com/en-us/news/us/transgender-a...,2023-06-04T15:39:22Z,\n\n\n\n\nMore On:\n\n\t\t\t\t\tsports media\t...,"TALLAHASSEE, Fla. (AP) Debate surrounding Flor..."
29677,31940,msnNOW,NaN,Jamie Foxx Left With A Brain Blood Cot After G...,https://www.msn.com/en-us/health/medical/jamie...,2023-06-01T19:14:33Z,Share to FacebookShare to TwitterShare to Link...,Jamie Foxxs bizarre health journey continues a...
29805,32070,msnNOW,NaN,UPDATE: Indian officials search for answers af...,https://www.msn.com/en-us/news/world/indias-wo...,2023-06-04T03:31:37Z,\n\n\n\n\nEntertainment\n\n\n\n\n\nTV\n\n\n\n\...,By Abinaya V and Jatindra Dash\r\n© Thomson Re...


## Initial (prior to preprocessing) Exploratory Data Analysis (EDA)

Perform only the EDA steps needed to preprocess the data, e.g., checking for missing values.

### Count missing `article_text` feature

The majority of null values appear in the `author` column. There are also two in `article_text`. The `author` feature will not be used for current modeling efforts, therefore it is not a factor. The instances with missing article text will be removed.

In [8]:
na_cnt = slct_tbl_full_df01.isnull().sum()
 
# Display missing values
print('Missing value counts:\n' + na_cnt.to_string())

Missing value counts:
text_id           0
source_name       0
author          216
title             0
url               0
publish_date      0
article_text      2
content           0


### Count blank `article_text` feature

Show instances with blank or missing `article_text` values; *n* = 2.

In [9]:
ic(len(slct_tbl_full_df01[slct_tbl_full_df01['article_text']==None]))
display(slct_tbl_full_df01[slct_tbl_full_df01['article_text'].isna()].head(23))

ic| len(slct_tbl_full_df01[slct_tbl_full_df01['article_text']==None]): 0


,text_id,source_name,author,title,url,publish_date,article_text,content
20850,22986,Fox News,Fox News Staff,"Mug shots of the week: June 11-17, 2023",https://www.foxnews.com/us/mug-shots-week-june...,2023-06-18T00:00:51Z,NaN,These mug shots were taken for arrests made th...
34174,66130,Fox News,Fox News Staff,"Mug shots of the week: June 18-24, 2023",https://www.foxnews.com/us/mug-shots-week-june...,2023-06-25T00:00:48Z,NaN,These mug shots were taken for arrests made th...


### Remove missing `article_text` row(s)

Use pandas' `dropna()` method to remove instances with null values; send results to a new dataframe. Print resulting df shape and head.

In [10]:
'''Drop missing citation:
https://pandas.pydata.org/pandas-docs/stable/reference
/api/pandas.DataFrame.dropna.html#pandas.DataFrame.dropna
'''
slct_tbl_full_df02 = slct_tbl_full_df01.dropna(subset=['article_text'])
slct_tbl_full_df04 = slct_tbl_full_df02.copy()
slct_tbl_full_df02 = slct_tbl_full_df02.loc[slct_tbl_full_df02['source_name'] != 'msnNOW']
print(f'Dataframe shape: {slct_tbl_full_df02.shape}')
display(slct_tbl_full_df02.head())

Dataframe shape: (36405, 8)


,text_id,source_name,author,title,url,publish_date,article_text,content
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...


### Display Source counts

Count the number of instances within each news source, e.g., ABC, CNN, USA Today.

In [11]:
source_cnt = slct_tbl_full_df02['source_name'].value_counts()
 
# Display missing values
print('Source counts:\n' + source_cnt.to_string())

source_cnt.to_csv(os.path.join(up1_dir,
                               ref_docs_location,
                               'source_dist.csv'))

Source counts:
Forbes                 7786
New York Post          6862
USA Today              6752
ABC News               4858
CNN                    1686
Business Insider       1563
Fox News               1379
CNBC                   1338
NBC News               1111
Breitbart News         1038
Buzzfeed                750
The Washington Post     374
PEOPLE                  245
MSNBC                   225
Vox                     213
Wired                   129
Reuters                  96


## Data preprocessing - Column 1: `article_text`; column 2: `source_name` and `author` combined

Define a function to display all individual tokens across an entire dataframe column.

In [12]:
def uniq_tok(df_col=pd.DataFrame([''])):
    '''Display all unique tokens across all instances'''
    start_time = time.perf_counter()
    df_cols1 = pd.Series(df_col)

    all_tokens_lst01 = []

    [all_tokens_lst01.append(f) for f in df_cols1]
    all_tokens_lst01 = list(itertools.chain\
                            .from_iterable(all_tokens_lst01))
    
    all_tokens_set01 = set(all_tokens_lst01)
    print(len(sorted(all_tokens_set01)))
    #print(sorted(all_tokens_set01))
    end_time = time.perf_counter()
    print(f'\nElapsed processing time = {end_time - start_time}')
    
    return sorted(all_tokens_set01)

Define a function to display example instance values.

In [13]:
def display_example(df=pd.DataFrame(),
                    df_iter_col='',
                    txt_col='',
                    iter_lst=[],
                    upper_rng=dply_rng_end01,
                    df_url_col='url',
                    df_auth_col='author',
                    df_title_col='title'
                   ):
    '''
    Show example values from dataframe column
    '''
    df01 = df.copy()
    
    '''Random number list citation:
    https://www.geeksforgeeks.org/generating-random-number-list-in-python/
    '''

    for s in iter_lst:
        ic()
        df01a = df01.loc[df01[df_iter_col] == s]
        df01a = df01a.reset_index()
        print(len(df01a))
        
        rand_lst = []
        n = upper_rng
        print(n)
        for i in range(n):
            rand_lst.append(random.randint(1,len(df01a)))
            rand_set = set(rand_lst)
            rand_lst = list(rand_set)
        print(rand_lst)
        
        counter = 0
        
        for c in rand_lst:
            ic()
            counter += 1
            print(f"'{s}' article #{counter}")
            try:
                print('URL:', df01a[df_url_col][c])
                print('Author(s):', df01a[df_auth_col][c])
                print('Title:', df01a[df_title_col][c])
                print(df01a[txt_col][c], '\n')
            except:
                print(f'Skip {c}', '\n')

Define a function to check number of tokens

In [14]:
def token_cnt(df=pd.DataFrame(),
              df_col=''):
    '''Raw text character and word counts'''
    df01 = df.copy()
    df01['char_cnt'] = df01[df_col]\
    .progress_apply(len)
    df01['num_tokens'] = df01[df_col]\
    .progress_apply(lambda x: len(x.split()))
    
    char_cnt = df01['char_cnt'].sum()
    token_cnt = df01['num_tokens'].sum()

    print('Total number of characters =', char_cnt)
    print('Total number of tokens =', token_cnt)
    print('Avg token len = ', char_cnt/token_cnt)
    display(df01.head())

Create fresh dataframe for applying preprocessing.

In [15]:
slct_tbl_full_df03 = slct_tbl_full_df02.copy()

Display unique `source_name` values.

In [16]:
source_name_lst = slct_tbl_full_df03['source_name'].unique()
ic(source_name_lst)
ic()

ic| source_name_lst: array(['USA Today', 'ABC News', 'CNN', 'Business Insider',
                            'Breitbart News', 'MSNBC', 'Buzzfeed', 'Wired', 'NBC News',
                            'The Washington Post', 'Fox News', 'Reuters', 'New York Post',
                            'Forbes', 'PEOPLE', 'CNBC', 'Vox'], dtype=object)
ic| 301998868.py:3 in <module> at 09:55:18.097


### Case-loading - columns 1 and 2

#### Apply

Combine `source_name` and `author` columns, then apply the `str.lower()` function to the `author_source` and `article_text` pandas dataframe columns.

In [17]:
slct_tbl_full_df03['article_lower'] = slct_tbl_full_df03['article_text']\
.progress_apply(str.lower)

slct_tbl_full_df03['author_source'] = \
slct_tbl_full_df03['source_name'] + ' ' + slct_tbl_full_df03['author']

'''Remove float citation:
https://chat.openai.com/share/dc596310-0c93-4542-9064-3034e85ed0a1
'''
slct_tbl_full_df03['author_source_lower'] = \
slct_tbl_full_df03['author_source']\
.progress_apply(lambda x: str(x).lower() if isinstance(x, str) else '')

100%|███| 36405/36405 [00:00<00:00, 702427.26it/s]


Display token count at current processing step.

In [18]:
token_cnt(df=slct_tbl_full_df03,
          df_col='article_lower')

100%|████| 36405/36405 [00:01<00:00, 25573.38it/s]

Total number of characters = 176865851
Total number of tokens = 28423163
Avg token len =  6.222595669595252


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,author_source_lower,char_cnt,num_tokens
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY","usa today usa today, emily deletter, usa today",1234,202
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY","usa today usa today, joy ashford, usa today",2310,346
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,usa today tar heels wire,2122,307
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,usa today roll tide wire,2821,404
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,usa today celtics wire,1867,245


### Clean `author_lower` and tokenize unique values - column 2

#### Create function

Replace comma with space, split on white space, and tokenize unqiue values.

In [19]:
def clean_author(text):
    text_p = text.replace(',', ' ')
    text_p = text_p.split(' ')
    text_p = set(text_p)
    text_p = list(text_p)
    
    return text_p

#### Call function

Apply custom function to entire text column of the dataframe.

In [20]:
slct_tbl_full_df03['author_source_lower_p'] = \
slct_tbl_full_df03['author_source_lower'].progress_apply(clean_author)

slct_tbl_full_df03['author_source_lower_p_join'] = \
slct_tbl_full_df03['author_source_lower_p'].progress_apply(" ".join)

100%|██| 36405/36405 [00:00<00:00, 1161344.97it/s]


Display shape and first few records of resulting dataframe.

In [21]:
ic(slct_tbl_full_df03.shape)
display(slct_tbl_full_df03.head())

ic| slct_tbl_full_df03.shape: (36405, 13)


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,author_source_lower,author_source_lower_p,author_source_lower_p_join
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY","usa today usa today, emily deletter, usa today","[usa, , deletter, today, emily]",usa deletter today emily
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY","usa today usa today, joy ashford, usa today","[usa, , joy, today, ashford]",usa joy today ashford
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,usa today tar heels wire,"[usa, heels, tar, wire, today]",usa heels tar wire today
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,usa today roll tide wire,"[usa, roll, tide, wire, today]",usa roll tide wire today
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,usa today celtics wire,"[usa, wire, celtics, today]",usa wire celtics today


#### Display globally unique tokens

Show unqiue tokens across entire `author_source_lower_p` column.

In [22]:
#print(uniq_tok(slct_tbl_full_df03['author_source_lower_p']))

### Text normalization - columns 1 and 2

#### Create function

Define function to normalize aspects of the text, including hyphens, quotation marks, and unicode; remove accent marks, where applicable.

In [23]:
def normalize(text):
    text_p = tprep.normalize.hyphenated_words(text)
    text_p = tprep.normalize.quotation_marks(text_p)
    text_p = tprep.normalize.unicode(text_p)
    text_p = tprep.remove.accents(text_p)
    
    return text_p

#### Call function

Apply custom function to entire text column of the dataframe.

In [24]:
slct_tbl_full_df03['article_norm'] = \
slct_tbl_full_df03['article_lower'].progress_apply(normalize)

'''-------------------------------------------------------------------------'''

slct_tbl_full_df03['author_source_norm'] = \
slct_tbl_full_df03['author_source_lower_p_join'].progress_apply(normalize)

100%|████| 36405/36405 [00:00<00:00, 75656.30it/s]


Print shape and first few records of resulting dataframe; also display a sample of full text.

In [25]:
ic(slct_tbl_full_df03.shape)
display(slct_tbl_full_df03.head())

ic| slct_tbl_full_df03.shape: (36405, 15)


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,author_source_lower,author_source_lower_p,author_source_lower_p_join,article_norm,author_source_norm
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY","usa today usa today, emily deletter, usa today","[usa, , deletter, today, emily]",usa deletter today emily,have you ever wanted to own your very own keg ...,usa deletter today emily
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY","usa today usa today, joy ashford, usa today","[usa, , joy, today, ashford]",usa joy today ashford,over five months after julian sands went missi...,usa joy today ashford
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,usa today tar heels wire,"[usa, heels, tar, wire, today]",usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,usa today roll tide wire,"[usa, roll, tide, wire, today]",usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,usa today celtics wire,"[usa, wire, celtics, today]",usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today


In [26]:
for c in range(0,dply_rng_end02):
    try:
        display(slct_tbl_full_df03['article_norm'][c],
                '\n')
    except:
        print(f'Skip {c}')

'have you ever wanted to own your very own keg of tito\'s handmade vodka? now, that dream can become a reality.the vodka brand is launching tito\'s in a big can, a $200, 128-ounce stainless steel mini cocktail keg. the keg includes an airtight cap, a co2 regulator, stainless steel spear, tap and handle and an extra set of o-rings. it comes empty, but tito\'s said it can be filled with their own vodka or with anything else.tito\'s is also releasing tito\'s in any can, a smaller insulated can-cooler for $30 that also functions as a refillable cup. it can fit both 16-ounce and 12-ounce cans, and converts into a refillable pint tumbler with a leak-proof lid."rebel against canformity," the product\'s description reads. "because we don\'t make canned cocktails, pre-mixes, or seltzers. you do."10best:which fast food restaurants will reign supreme? cast your votes nowwhat are americans drinking this summer?tequila, modelo, canned cocktails and non-alcoholic drinksboth products are available fo

'\n'

'over five months after julian sands went missing during a solo hike in california, local police have announced the ongoing search for the actor will continue in a "limited capacity."the san bernardino county sheriff\'s office released a statement monday on the department\'s continued efforts to locate the "a room with a view" actor.sands, 65, was reported missing on jan. 13 while hiking a trail on mount baldy in southern california, san bernardino county sheriff\'s department spokeswoman gloria huerta said at the time. authorities\' search efforts, both on the ground and by air, have been paused and resumed based on a series of storms that has created dangerous mountain conditions in the area.on saturday, the sheriff\'s office conducted another search for sands involving over 80 search and rescue volunteers, deputies and staff. two helicopters helped search teams reach remote areas of mount baldy, and drone crews searched areas inaccessible to hikers on the ground.despite the departme

'\n'

Display token count at current processing step.

In [27]:
token_cnt(df=slct_tbl_full_df03,
          df_col='article_norm')

100%|████| 36405/36405 [00:01<00:00, 24911.86it/s]

Total number of characters = 176879054
Total number of tokens = 28421854
Avg token len =  6.223346795040182


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,author_source_lower,author_source_lower_p,author_source_lower_p_join,article_norm,author_source_norm,char_cnt,num_tokens
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY","usa today usa today, emily deletter, usa today","[usa, , deletter, today, emily]",usa deletter today emily,have you ever wanted to own your very own keg ...,usa deletter today emily,1234,202
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY","usa today usa today, joy ashford, usa today","[usa, , joy, today, ashford]",usa joy today ashford,over five months after julian sands went missi...,usa joy today ashford,2310,346
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,usa today tar heels wire,"[usa, heels, tar, wire, today]",usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,2122,307
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,usa today roll tide wire,"[usa, roll, tide, wire, today]",usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,2821,404
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,usa today celtics wire,"[usa, wire, celtics, today]",usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,1867,245


### Remove/clean source/website specific artifacts using regular expressions, Phase 1: Remove large phrases and decimals; collapse acronyms (i.e., remove delimiters) - column 1

#### Create function

Define function to eliminate specific website artifacts that add noise to the data.

In [28]:
rex_like_article = rex.compile(r'''like this article\?[\n]+sign[(up)|(for)|(the)|\s]+\s[\w\s]*newsletter to get our top stories in your inbox every morning[\n,\s,\t]''')
#rex_click_here = rex.compile(r'click here to sign up for[(our)|(the)\s]+\s[\w\s]*newsletter')
rex_click_here = rex.compile(r'click(?:here|to|get|ge|the|for|sign|read|up|our|hear|more|instagram|twitter|view|learn|\s)*[\w\s]*(?:newsletter|app|reporting|cartoons){1}\b')
rex_copyright = rex.compile(r'copyright\s[\d]+[\w\s]*[^\n]+[\.|\n]{1}')
rex_sign_up = rex.compile(r'email[\n|\t|\s]+sign up[\n|\t|\s]+view all newsletters[\n|\t|\s]+more[\w\s]+[\n|\t|\s]{1}')
rex_gamble = rex.compile(r'gannett[\S|\s|]*gamble\.')
rex_tipco = rex.compile(r'catch the excitement[\S|\s|\n]+conditions.')
rex_tc = rex.compile(r'must be\s[\d]+[\S|\s]+apply\.')
rex_elipses = rex.compile(r'[\.]{3}')
rex_dec = rex.compile(r'\b\d+\.\d+\b')
rex_acronym = rex.compile(r'(?:[a-z]\.){2,}')

rex_forbes = rex.compile(r'\b[\s]*more[\s]*from[\s]forbes')
rex_more_by = rex.compile(r'\b[\s]*moreby[\w|\s]*(?:press|staff)+')
rex_gif = rex.compile(r'\b[\s]*tap to play gif[\s]*\b')

'''re.sub lambda citation:
https://chat.openai.com/share/402ec66e-2802-4cda-af8c-6f9f5b097d85
'''
def remove_rex(text):
    repl_text = rex_like_article.sub(r' ', text)
    repl_text = rex_click_here.sub(r' ', repl_text)
    repl_text = rex_copyright.sub(r' ', repl_text)
    repl_text = rex_sign_up.sub(r' ', repl_text)
    repl_text = rex_gamble.sub(r' ', repl_text)
    repl_text = rex_tipco.sub(r' ', repl_text)
    repl_text = rex_tc.sub(r' ', repl_text)
    repl_text = rex_elipses.sub(r' ', repl_text)
    repl_text = rex_dec.sub(lambda match: match.group().replace('.', ''),
                            repl_text)
    repl_text = rex_acronym.sub(lambda match: match.group().replace('.', ''),
                                repl_text)
    repl_text = rex_forbes.sub(r' ', repl_text)
    repl_text = rex_more_by.sub(r' ', repl_text)
    repl_text = rex_gif.sub(r' ', repl_text)
    
    return repl_text

#### Call function

Apply custom function to entire text column of the dataframe.

In [29]:
slct_tbl_full_df03['article_rex_trimmed'] = \
slct_tbl_full_df03['article_norm'].progress_apply(remove_rex)

100%|█████| 36405/36405 [00:09<00:00, 3965.92it/s]


Print shape and first few records of resulting dataframe; also display a sample of full text.

In [30]:
ic(slct_tbl_full_df03.shape)
display(slct_tbl_full_df03.head())

ic| slct_tbl_full_df03.shape: (36405, 16)


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,author_source_lower,author_source_lower_p,author_source_lower_p_join,article_norm,author_source_norm,article_rex_trimmed
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY","usa today usa today, emily deletter, usa today","[usa, , deletter, today, emily]",usa deletter today emily,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY","usa today usa today, joy ashford, usa today","[usa, , joy, today, ashford]",usa joy today ashford,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,usa today tar heels wire,"[usa, heels, tar, wire, today]",usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,usa today roll tide wire,"[usa, roll, tide, wire, today]",usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,usa today celtics wire,"[usa, wire, celtics, today]",usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...


In [31]:
display_example(df=slct_tbl_full_df03,
                df_iter_col='source_name',
                txt_col='article_rex_trimmed',
                iter_lst=source_name_lst,
                upper_rng=dply_rng_end01
               )

ic| 1699879310.py:20 in display_example() at 09:56:23.275
ic| 1699879310.py:20 in display_example() at 09:56:23.304
ic| 1699879310.py:20 in display_example() at 09:56:23.333
ic| 1699879310.py:20 in display_example() at 09:56:23.360
ic| 1699879310.py:20 in display_example() at 09:56:23.394
ic| 1699879310.py:20 in display_example() at 09:56:23.420
ic| 1699879310.py:20 in display_example() at 09:56:23.442
ic

6752
0
[]
4858
0
[]
1686
0
[]
1563
0
[]
1038
0
[]
225
0
[]
750
0
[]


| 1699879310.py:20 in display_example() at 09:56:23.469
ic| 1699879310.py:20 in display_example() at 09:56:23.504
ic| 1699879310.py:20 in display_example() at 09:56:23.540
ic| 1699879310.py:20 in display_example() at 09:56:23.567
ic| 1699879310.py:20 in display_example() at 09:56:23.591
ic| 1699879310.py:20 in display_example() at 09:56:23.619
ic|

129
0
[]
1111
0
[]
374
0
[]
1379
0
[]
96
0
[]
6862
0
[]


 1699879310.py:20 in display_example() at 09:56:23.660
ic| 1699879310.py:20 in display_example() at 09:56:23.705
ic| 1699879310.py:20 in display_example() at 09:56:23.737
ic| 1699879310.py:20 in display_example() at 09:56:23.771


7786
0
[]
245
0
[]
1338
0
[]
213
0
[]


Display token count at current processing step.

In [32]:
token_cnt(df=slct_tbl_full_df03,
          df_col='article_rex_trimmed')

100%|████| 36405/36405 [00:01<00:00, 20680.10it/s]

Total number of characters = 174857710
Total number of tokens = 28130846
Avg token len =  6.215871005088151


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,author_source_lower,author_source_lower_p,author_source_lower_p_join,article_norm,author_source_norm,article_rex_trimmed,char_cnt,num_tokens
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY","usa today usa today, emily deletter, usa today","[usa, , deletter, today, emily]",usa deletter today emily,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,1234,202
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY","usa today usa today, joy ashford, usa today","[usa, , joy, today, ashford]",usa joy today ashford,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,2310,346
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,usa today tar heels wire,"[usa, heels, tar, wire, today]",usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,1920,278
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,usa today roll tide wire,"[usa, roll, tide, wire, today]",usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,2611,375
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,usa today celtics wire,"[usa, wire, celtics, today]",usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,1758,225


### Remove special characters - columns 1 and 2

#### Create function

Define function to eliminate unicode characters and replace hyphens with a space.

In [33]:
def str_replace(text):
    text_p = text.replace(r'&nbsp;', ' ')\
    .replace(r'-', ' ').replace(r'\u2060', ' ')\
    .replace('\u2063', ' ').replace('\u2066', ' ')\
    .replace('\u2069', ' ').replace('\u200b', ' ')\
    .replace('\u200d', ' ').replace('\ufeff', ' ')\
    .replace('\u200c', ' ').replace('\u200e', ' ')\
    .replace('\u200f', ' ').replace('\xad', ' ')

    return text_p

#### Call function

Apply custom function to entire text column of the dataframe.

In [34]:
slct_tbl_full_df03['article_trimmed'] = \
slct_tbl_full_df03['article_rex_trimmed'].progress_apply(str_replace)

'''-------------------------------------------------------------------------'''

slct_tbl_full_df03['author_source_trimmed'] = \
slct_tbl_full_df03['author_source_norm'].progress_apply(str_replace)

100%|███| 36405/36405 [00:00<00:00, 576829.30it/s]


Print shape and first few records of resulting dataframe.

In [35]:
ic(slct_tbl_full_df03.shape)
display(slct_tbl_full_df03.head())

ic| slct_tbl_full_df03.shape: (36405, 18)


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,author_source_lower,author_source_lower_p,author_source_lower_p_join,article_norm,author_source_norm,article_rex_trimmed,article_trimmed,author_source_trimmed
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY","usa today usa today, emily deletter, usa today","[usa, , deletter, today, emily]",usa deletter today emily,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY","usa today usa today, joy ashford, usa today","[usa, , joy, today, ashford]",usa joy today ashford,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,usa today tar heels wire,"[usa, heels, tar, wire, today]",usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,usa today roll tide wire,"[usa, roll, tide, wire, today]",usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,usa today celtics wire,"[usa, wire, celtics, today]",usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today


Display token count at current processing step.

In [36]:
token_cnt(df=slct_tbl_full_df03,
          df_col='article_trimmed')

100%|████| 36405/36405 [00:01<00:00, 21919.98it/s]

Total number of characters = 174818910
Total number of tokens = 28554715
Avg token len =  6.122243209221314


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,author_source_lower,author_source_lower_p,author_source_lower_p_join,article_norm,author_source_norm,article_rex_trimmed,article_trimmed,author_source_trimmed,char_cnt,num_tokens
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY","usa today usa today, emily deletter, usa today","[usa, , deletter, today, emily]",usa deletter today emily,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,1234,210
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY","usa today usa today, joy ashford, usa today","[usa, , joy, today, ashford]",usa joy today ashford,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,2310,346
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,usa today tar heels wire,"[usa, heels, tar, wire, today]",usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,1920,290
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,usa today roll tide wire,"[usa, roll, tide, wire, today]",usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,2611,397
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,usa today celtics wire,"[usa, wire, celtics, today]",usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,1758,235


### Separate emojis as individual tokens - column 1

#### Create function

Define function to add a blank space between individual emojis and other text.

In [37]:
def emoji_split(text):
    #text_p = "".join([' ' + c + ' '\
    #                  if emoji.is_emoji(c) else c for c in text])
    
    #return text_p
    return("".join([' ' + c + ' ' if emoji.is_emoji(c) else c for c in text]))

#### Call function

Apply custom function to entire text column of the dataframe.

In [38]:
slct_tbl_full_df03['article_sep_emoji'] = \
slct_tbl_full_df03['article_trimmed'].progress_apply(emoji_split)

100%|█████| 36405/36405 [00:35<00:00, 1014.07it/s]


Print shape and first few records of resulting dataframe; also display a sample of full text.

In [39]:
ic(slct_tbl_full_df03.shape)
display(slct_tbl_full_df03.head())

ic| slct_tbl_full_df03.shape: (36405, 19)


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,author_source_lower,author_source_lower_p,author_source_lower_p_join,article_norm,author_source_norm,article_rex_trimmed,article_trimmed,author_source_trimmed,article_sep_emoji
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY","usa today usa today, emily deletter, usa today","[usa, , deletter, today, emily]",usa deletter today emily,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY","usa today usa today, joy ashford, usa today","[usa, , joy, today, ashford]",usa joy today ashford,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,usa today tar heels wire,"[usa, heels, tar, wire, today]",usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,usa today roll tide wire,"[usa, roll, tide, wire, today]",usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,usa today celtics wire,"[usa, wire, celtics, today]",usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...


In [40]:
for c in range(0,dply_rng_end01):
    try:
        print(slct_tbl_full_df03['article_sep_emoji'][c],
              '\n')
    except:
        print(f'Skip {c}')

Display token count at current processing step.

In [41]:
token_cnt(df=slct_tbl_full_df03,
          df_col='article_sep_emoji')

100%|████| 36405/36405 [00:01<00:00, 21974.47it/s]

Total number of characters = 174832184
Total number of tokens = 28560113
Avg token len =  6.1215508496062325


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,...,author_source_lower_p,author_source_lower_p_join,article_norm,author_source_norm,article_rex_trimmed,article_trimmed,author_source_trimmed,article_sep_emoji,char_cnt,num_tokens
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,"[usa, , deletter, today, emily]",usa deletter today emily,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,1234,210
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,"[usa, , joy, today, ashford]",usa joy today ashford,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,2310,346
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,"[usa, heels, tar, wire, today]",usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,1922,290
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,"[usa, roll, tide, wire, today]",usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,2611,397
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,"[usa, wire, celtics, today]",usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,1758,235


### Remove URLs, hash tags, and handles - columns 1 and 2

#### Create function

Define function to using regular expressions to remove URLs, hash tags, and Twitter handles.

In [42]:
#rex_url_c = rex.compile(r'http[s]?:[\/]+[\S]*\s')
rex_url_c = rex.compile(r'(?:http|https|www)+[\.|\S]+\b')
rex_hasht = rex.compile(r'\b#[\S]+\b')
rex_handle = rex.compile(r'[^\.|!|\?|\"|\s]+\@[\S]+\b')

'''re.sub lambda citation:
https://chat.openai.com/share/402ec66e-2802-4cda-af8c-6f9f5b097d85
'''
# Add leading and trailing space to URLs
def remove_url(text):
    text_p = rex_url_c.sub(r' ', text)
    text_p = rex_hasht.sub(r' ', text_p)
    text_p = rex_handle.sub(r' ', text_p)
    
    return text_p

#### Call function

Apply custom function to entire text column of the dataframe.

In [43]:
slct_tbl_full_df03['article_urls_removed'] = \
slct_tbl_full_df03['article_sep_emoji'].progress_apply(remove_url)

'''-------------------------------------------------------------------------'''

slct_tbl_full_df03['author_source_urls_removed'] = \
slct_tbl_full_df03['author_source_trimmed'].progress_apply(remove_url)

100%|███| 36405/36405 [00:00<00:00, 299946.84it/s]


Print shape and first few records of resulting dataframe; also display a sample of full text.

In [44]:
ic(slct_tbl_full_df03.shape)
display(slct_tbl_full_df03.head())

ic| slct_tbl_full_df03.shape: (36405, 21)


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,...,author_source_lower_p,author_source_lower_p_join,article_norm,author_source_norm,article_rex_trimmed,article_trimmed,author_source_trimmed,article_sep_emoji,article_urls_removed,author_source_urls_removed
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,"[usa, , deletter, today, emily]",usa deletter today emily,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,"[usa, , joy, today, ashford]",usa joy today ashford,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,"[usa, heels, tar, wire, today]",usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,"[usa, roll, tide, wire, today]",usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,"[usa, wire, celtics, today]",usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today


In [45]:
display_example(df=slct_tbl_full_df03,
                df_iter_col='source_name',
                txt_col='article_urls_removed',
                iter_lst=source_name_lst,
                upper_rng=dply_rng_end01
               )

ic| 1699879310.py:20 in display_example() at 09:57:11.210
ic| 1699879310.py:20 in display_example() at 09:57:11.249
ic| 1699879310.py:20 in display_example() at 09:57:11.280
ic| 1699879310.py:20 in display_example() at 09:57:11.299
ic| 1699879310.py:20 in display_example() at 09:57:11.329
ic| 1699879310.py:20 in display_example() at 09:57:11.349
ic| 1699879310.py:20 in display_example() at 09:57:11.380
ic| 1699879310.py:

6752
0
[]
4858
0
[]
1686
0
[]
1563
0
[]
1038
0
[]
225
0
[]
750
0
[]


20 in display_example() at 09:57:11.413
ic| 1699879310.py:20 in display_example() at 09:57:11.440
ic| 1699879310.py:20 in display_example() at 09:57:11.467
ic| 1699879310.py:20 in display_example() at 09:57:11.489
ic| 1699879310.py:20 in display_example() at 09:57:11.514
ic| 1699879310.py:20 in display_example() at 09:57:11.539
ic| 1699879310.py:20 in display_example() at 09:57:11.583


129
0
[]
1111
0
[]
374
0
[]
1379
0
[]
96
0
[]
6862
0
[]


ic| 1699879310.py:20 in display_example() at 09:57:11.619
ic| 1699879310.py:20 in display_example() at 09:57:11.649
ic| 1699879310.py:20 in display_example() at 09:57:11.673


7786
0
[]
245
0
[]
1338
0
[]
213
0
[]


Display token count at current processing step.

In [46]:
token_cnt(df=slct_tbl_full_df03,
          df_col='article_urls_removed')

100%|████| 36405/36405 [00:01<00:00, 23531.09it/s]

Total number of characters = 174005717
Total number of tokens = 28545329
Avg token len =  6.095768488077331


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,...,article_norm,author_source_norm,article_rex_trimmed,article_trimmed,author_source_trimmed,article_sep_emoji,article_urls_removed,author_source_urls_removed,char_cnt,num_tokens
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,1234,210
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,2310,346
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,1859,289
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,2547,396
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,1645,231


### Remove/clean source/website specific artifacts useing regular expressions, Phase 2: Remove returns, tabs, and additional phrases; eliminate amazon ad text - column 1

#### Create function

Define function to remove multiple instances of returns, tabs, and spaces; also remove digits.

In [47]:
rex_tabs = rex.compile(r'[\t]+')
rex_returns = rex.compile(r'[\n]+')
rex_errant_returns = rex.compile(r'[\n][\s]+[\n]')
rex_numbers = rex.compile(r'[\d]+')
rex_ja = rex.compile(r'jason[\s]*abbruzzese')
rex_amazon = rex.compile(r'amazon[s]*(?:ian)*')
rex_acct = rex.compile(r'(?:un)*account(?:ing|ability|able|ant|ancy|ed|ably)*(?:s)?')
rex_ads = rex.compile(r'advertisement(?:s)*')
rex_contrib = rex.compile(r'contributorfacebookpinteresttwittermaillink')
rex_share = rex.compile(r'share to(?:facebook|share|to|twitter|linkedin|\s)+')
rex_bade = rex.compile(r'by\s*amy\s*danise\s*editor')
rex_wir = rex.compile(r'explainerinformationwired')
rex_getty = rex.compile(r'\b(?:photo|by|\s)*[\w\s]*getty[\s]*images[\s]*')
rex_amz1 = rex.compile(r'\.[\w|\s]*amazon[\s*]check[\w|\s]*\.')
rex_amz2 = rex.compile(r'\.\n+\s*amazon[\w|\s]*\n+\s*\.')
rex_amz3 = rex.compile(r'\.[\s]*amazon[\w\s]*small(?:biz|business|\s)+[\w\s]*\.')
rex_amz4 = rex.compile(r'\.\n+\s*shop\s*amazon[\w|\s]*(?:\s\n)+')
rex_amz5 = rex.compile(r'\.\s*(?:buy|sign up|you can)+[\w\s]*amazon[\w|\s]*\.')
rex_multispaces = rex.compile(r'[ ]{2,}')

def remove_returns(text):
    repl_text = rex_tabs.sub(r' ', text)
    repl_text = rex_returns.sub(r'\n', repl_text)
    repl_text = rex_errant_returns.sub(r'\n', repl_text)
    repl_text = rex_numbers.sub(r' ', repl_text)
    repl_text = rex_ja.sub(lambda match: ' ' + match.group(0) + ' ',
                           repl_text)
    repl_text = rex_amazon.sub(lambda match: ' ' + match.group(0) + ' ',
                               repl_text)
    repl_text = rex_acct.sub(lambda match: ' ' + match.group(0) + ' ',
                             repl_text)
    repl_text = rex_ads.sub(lambda match: ' ' + match.group(0) + ' ',
                            repl_text)
    repl_text = rex_contrib.sub(r' ', repl_text)
    repl_text = rex_share.sub(r' ', repl_text)
    repl_text = rex_bade.sub(r' ', repl_text)
    repl_text = rex_wir.sub(r' ', repl_text)
    repl_text = rex_getty.sub(r' ', repl_text)
    repl_text = rex_amz1.sub(r' ', repl_text)
    repl_text = rex_amz2.sub(r' ', repl_text)
    repl_text = rex_amz3.sub(r' ', repl_text)
    repl_text = rex_amz4.sub(r' ', repl_text)
    repl_text = rex_amz5.sub(r' ', repl_text)
    repl_text = rex_multispaces.sub(r' ', repl_text)

    return repl_text

#### Call function

Apply custom function to entire text column of the dataframe.

In [48]:
slct_tbl_full_df03['article_w_one_return'] = \
slct_tbl_full_df03['article_urls_removed'].progress_apply(remove_returns)

100%|█████| 36405/36405 [00:16<00:00, 2236.86it/s]


Print shape and first few records of resulting dataframe; also display a sample of full text.

In [49]:
ic(slct_tbl_full_df03.shape)
display(slct_tbl_full_df03.head())

ic| slct_tbl_full_df03.shape: (36405, 22)


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,...,author_source_lower_p_join,article_norm,author_source_norm,article_rex_trimmed,article_trimmed,author_source_trimmed,article_sep_emoji,article_urls_removed,author_source_urls_removed,article_w_one_return
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,usa deletter today emily,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,usa joy today ashford,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\nfour star running back picks michigan state ...
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\nalabama center charles bediako signs one yea...
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\nralph sampson breaks down i

In [50]:
display_example(df=slct_tbl_full_df03,
                df_iter_col='source_name',
                txt_col='article_w_one_return',
                iter_lst=source_name_lst,
                upper_rng=dply_rng_end01
               )

ic| 1699879310.py:20 in display_example() at 09:57:29.784
ic| 1699879310.py:20 in display_example() at 09:57:29.810
ic| 1699879310.py:20 in display_example() at 09:57:29.840
ic| 1699879310.py:20 in display_example() at 09:57:29.870
ic| 1699879310.py:20 in display_example() at 09:57:29.889
ic| 1699879310.py:20 in display_example() at 09:57:29.920
ic| 1699879310.py:20 in display_example() at 09:57:29.940
ic| 1699879310.py:20 in display_example() at 09:57:29.961


6752
0
[]
4858
0
[]
1686
0
[]
1563
0
[]
1038
0
[]
225
0
[]
750
0
[]
129
0
[]


ic| 1699879310.py:20 in display_example() at 09:57:29.988
ic| 1699879310.py:20 in display_example() at 09:57:30.019
ic| 1699879310.py:20 in display_example() at 09:57:30.047
ic| 1699879310.py:20 in display_example() at 09:57:30.070
ic| 1699879310.py:20 in display_example() at 09:57:30.094
ic| 1699879310.py:20 in display_example() at 09:57:30.139
ic| 1699879310.py:20 in display_example() at 09:57:30.178


1111
0
[]
374
0
[]
1379
0
[]
96
0
[]
6862
0
[]
7786
0
[]


ic| 1699879310.py:20 in display_example() at 09:57:30.199
ic| 1699879310.py:20 in display_example() at 09:57:30.229


245
0
[]
1338
0
[]
213
0
[]


Display token count at current processing step.

In [51]:
token_cnt(df=slct_tbl_full_df03,
          df_col='article_w_one_return')

100%|████| 36405/36405 [00:01<00:00, 24737.70it/s]

Total number of characters = 168345073
Total number of tokens = 28236864
Avg token len =  5.9618898543407655


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,...,author_source_norm,article_rex_trimmed,article_trimmed,author_source_trimmed,article_sep_emoji,article_urls_removed,author_source_urls_removed,article_w_one_return,char_cnt,num_tokens
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,1218,209
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,2270,341
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\nfour star running back picks michigan state ...,1631,283
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\nalabama center charles bediako signs one yea...,2278,379
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\nralph sampson breaks down iconic boston celt...,1389,227


### Pad End-of-Sentence (EOS) punctuation ("?", ".", and "!") - column 1

#### Create function

Define function to pad EOS punctuation.

In [52]:
def pad_eos_punc(text):
    text_p = text.replace(r'.', ' . ').replace(r'!', ' . ')\
    .replace(r'?', ' . ')
    
    return text_p

#### Call function

Apply custom function to entire text column of the dataframe.

In [53]:
slct_tbl_full_df03['article_punc_padded'] = \
slct_tbl_full_df03['article_w_one_return'].progress_apply(pad_eos_punc)

100%|████| 36405/36405 [00:00<00:00, 93239.20it/s]


Print shape and first few records of resulting dataframe; also display a sample of full text.

In [54]:
ic(slct_tbl_full_df03.shape)
display(slct_tbl_full_df03.head())

ic| slct_tbl_full_df03.shape: (36405, 23)


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,...,article_norm,author_source_norm,article_rex_trimmed,article_trimmed,author_source_trimmed,article_sep_emoji,article_urls_removed,author_source_urls_removed,article_w_one_return,article_punc_padded
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\nfour star running back picks michigan state ...,\nfour star running back picks michigan state ...
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\nalabama center charles bediako signs one yea...,\nalabama center charles bediako signs one yea...
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nr

In [55]:
display_example(df=slct_tbl_full_df03,
                df_iter_col='source_name',
                txt_col='article_punc_padded',
                iter_lst=source_name_lst,
                upper_rng=dply_rng_end01
               )

ic| 1699879310.py:20 in display_example() at 09:57:32.384
ic| 1699879310.py:20 in display_example() at 09:57:32.415
ic| 1699879310.py:20 in display_example() at 09:57:32.449
ic| 1699879310.py:20 in display_example() at 09:57:32.474
ic| 1699879310.py:20 in display_example() at 09:57:32.508
ic| 1699879310.py:20 in display_example() at 09:57:32.529
ic| 1699879310.py:20 in display_example() at 09:57:32.559
ic

6752
0
[]
4858
0
[]
1686
0
[]
1563
0
[]
1038
0
[]
225
0
[]
750
0
[]


| 1699879310.py:20 in display_example() at 09:57:32.580
ic| 1699879310.py:20 in display_example() at 09:57:32.602
ic| 1699879310.py:20 in display_example() at 09:57:32.630
ic| 1699879310.py:20 in display_example() at 09:57:32.654
ic| 1699879310.py:20 in display_example() at 09:57:32.681
ic| 1699879310.py:20 in display_example() at 09:57:32.711
ic| 1699879310.py:20 in display_example() at 09:57:32.739
ic

129
0
[]
1111
0
[]
374
0
[]
1379
0
[]
96
0
[]
6862
0
[]
7786
0
[]


| 1699879310.py:20 in display_example() at 09:57:32.776
ic| 1699879310.py:20 in display_example() at 09:57:32.800
ic| 1699879310.py:20 in display_example() at 09:57:32.824


245
0
[]
1338
0
[]
213
0
[]


Display token count at current processing step.

In [56]:
token_cnt(df=slct_tbl_full_df03,
          df_col='article_punc_padded')

100%|████| 36405/36405 [00:01<00:00, 22840.57it/s]

Total number of characters = 171202949
Total number of tokens = 29847528
Avg token len =  5.735917192204326


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,...,article_rex_trimmed,article_trimmed,author_source_trimmed,article_sep_emoji,article_urls_removed,author_source_urls_removed,article_w_one_return,article_punc_padded,char_cnt,num_tokens
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,1246,228
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...,2304,367
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\nfour star running back picks michigan state ...,\nfour star running back picks michigan state ...,1655,297
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\nalabama center charles bediako signs one yea...,\nalabama center charles bediako signs one yea...,2320,399
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\nralph sampson breaks down iconic boston celt...,\nralph sampson breaks down iconic boston celt...,1401,233


### Remove punctuation - columns 1 and 2

Customize string.punctuation list based on dataset specific values.

In [57]:
punctuation = set(punctuation) # speeds up comparison
#print(punctuation)

# Customize punctuation set
#tw_punct = punctuation - {"#"}
#tw_punct = punctuation - {"@"}
#tw_punct = punctuation - {"-"}
tw_punct = punctuation - {"."}
tw_punct = tw_punct - {"?"}
tw_punct = tw_punct - {"!"}
#tw_punct = tw_punct - {"/"}
tw_punct.add("’")
tw_punct.add("‘")
tw_punct.add("”")
tw_punct.add("“")
tw_punct.add("…")
tw_punct.add("—")
tw_punct.add("–")
tw_punct.add("...")
tw_punct.add("€")
tw_punct.add("±")
tw_punct.add("£")
tw_punct.add("¡")
tw_punct.add("§")
tw_punct.add("⦿")
tw_punct.add("➤")
tw_punct.add("►")
tw_punct.add("′")
tw_punct.add("•")
tw_punct.add("„")
tw_punct.add("¢")
tw_punct.add("¥")
tw_punct.add("«")
tw_punct.add("»")
tw_punct.add("†")
tw_punct.add("∎")

print(len(tw_punct))
print(tw_punct)

54
{'►', '...', '†', '\\', '^', '_', '%', '′', '§', ',', '—', '«', '¥', "'", '=', ')', '/', '@', '+', '*', '<', '£', '`', '…', '±', '~', '-', '|', '➤', '⦿', '}', '¡', '‘', ']', '„', '&', '”', '>', '¢', ':', '{', '#', '’', '–', '[', '(', '•', '“', '»', '"', ';', '∎', '$', '€'}


#### Create function

Define function to remove punctuation in custom set.

In [58]:
def remove_punctuation(text,
                       punct_set=tw_punct):
    #text_p = "".join([c for c in text if c not in punct_set])
    
    #return text_p
    return("".join([ch for ch in text if ch not in punct_set]))

#### Call function

Apply custom function to entire text column of the dataframe.

In [59]:
slct_tbl_full_df03['article_no_punc'] = \
slct_tbl_full_df03['article_punc_padded']\
.progress_apply(remove_punctuation,
                punct_set=tw_punct)

'''-------------------------------------------------------------------------'''

slct_tbl_full_df03['author_source_no_punc'] = \
slct_tbl_full_df03['author_source_urls_removed']\
.progress_apply(remove_punctuation,
                punct_set=tw_punct)

100%|███| 36405/36405 [00:00<00:00, 302448.68it/s]


Print shape and first few records of resulting dataframe; also display a sample of full text.

In [60]:
ic(slct_tbl_full_df03.shape)
display(slct_tbl_full_df03.head())

ic| slct_tbl_full_df03.shape: (36405, 25)


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,...,article_rex_trimmed,article_trimmed,author_source_trimmed,article_sep_emoji,article_urls_removed,author_source_urls_removed,article_w_one_return,article_punc_padded,article_no_punc,author_source_no_punc
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\nfour star running back picks michigan state ...,\nfour star running back picks michigan state ...,\nfour star running back picks michigan state ...,usa heels tar wire today
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\nalabama center charles bediako signs one yea...,\nalabama center charles bediako signs one yea...,\nalabama center charles bediako signs one yea...,usa roll tide wire today
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\nralph sampson breaks 

In [61]:
display_example(df=slct_tbl_full_df03,
                df_iter_col='source_name',
                txt_col='article_no_punc',
                iter_lst=source_name_lst,
                upper_rng=dply_rng_end01
               )

ic| 1699879310.py:20 in display_example() at 09:57:43.036
ic| 1699879310.py:20 in display_example() at 09:57:43.069
ic| 1699879310.py:20 in display_example() at 09:57:43.099
ic| 1699879310.py:20 in display_example() at 09:57:43.121
ic| 1699879310.py:20 in display_example() at 09:57:43.148
ic| 1699879310.py:20 in display_example() at 09:57:43.169
ic| 1699879310.py:20 in display_example() at 09:57:43.195
ic| 1699879310.py:20 in display_example() at 09:57:43.219
ic| 1699879310.py:20 in

6752
0
[]
4858
0
[]
1686
0
[]
1563
0
[]
1038
0
[]
225
0
[]
750
0
[]
129
0
[]


 display_example() at 09:57:43.241
ic| 1699879310.py:20 in display_example() at 09:57:43.265
ic| 1699879310.py:20 in display_example() at 09:57:43.291
ic| 1699879310.py:20 in display_example() at 09:57:43.321
ic| 1699879310.py:20 in display_example() at 09:57:43.340
ic| 1699879310.py:20 in display_example() at 09:57:43.378
ic| 1699879310.py:20 in display_example() at 09:57:43.411
ic

1111
0
[]
374
0
[]
1379
0
[]
96
0
[]
6862
0
[]
7786
0
[]
245
0
[]


| 1699879310.py:20 in display_example() at 09:57:43.439
ic| 1699879310.py:20 in display_example() at 09:57:43.465


1338
0
[]
213
0
[]


Display token count at current processing step.

In [62]:
token_cnt(df=slct_tbl_full_df03,
          df_col='article_no_punc')

100%|████| 36405/36405 [00:01<00:00, 21763.90it/s]

Total number of characters = 167828953
Total number of tokens = 29270687
Avg token len =  5.733686845136228


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,...,author_source_trimmed,article_sep_emoji,article_urls_removed,author_source_urls_removed,article_w_one_return,article_punc_padded,article_no_punc,author_source_no_punc,char_cnt,num_tokens
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,1212,224
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,2229,366
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\nfour star running back picks michigan state ...,\nfour star running back picks michigan state ...,\nfour star running back picks michigan state ...,usa heels tar wire today,1625,291
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\nalabama center charles bediako signs one yea...,\nalabama center charles bediako signs one yea...,\nalabama center charles bediako signs one yea...,usa roll tide wire today,2297,394
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\nralph sampson breaks down iconic boston celt...,\nralph sampson breaks down iconic boston celt...,\nralph sampson breaks down iconic boston celt...,usa wire celtics today,1378,230


### Case-loading - column 1

#### Apply

Combine `source_name` and `author` columns, then apply the `str.lower()` function to the `author_source` and `article_text` pandas dataframe columns.

In [63]:
slct_tbl_full_df03['article_no_punc_lower'] = slct_tbl_full_df03['article_no_punc']\
.progress_apply(str.lower)

100%|███| 36405/36405 [00:00<00:00, 122065.46it/s]


Display token count at current processing step.

In [64]:
token_cnt(df=slct_tbl_full_df03,
          df_col='article_no_punc_lower')

100%|████| 36405/36405 [00:01<00:00, 19245.26it/s]

Total number of characters = 167828953
Total number of tokens = 29270687
Avg token len =  5.733686845136228


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,...,article_sep_emoji,article_urls_removed,author_source_urls_removed,article_w_one_return,article_punc_padded,article_no_punc,author_source_no_punc,article_no_punc_lower,char_cnt,num_tokens
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,1212,224
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,2229,366
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\nfour star running back picks michigan state ...,\nfour star running back picks michigan state ...,\nfour star running back picks michigan state ...,usa heels tar wire today,\nfour star running back picks michigan state ...,1625,291
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\nalabama center charles bediako signs one yea...,\nalabama center charles bediako signs one yea...,\nalabama center charles bediako signs one yea...,usa roll tide wire today,\nalabama center charles bediako signs one yea...,2297,394
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\nralph sampson breaks down iconic boston celt...,\nralph sampson breaks down iconic boston celt...,\nralph sampson breaks down iconic boston celt...,usa wire celtics today,\nralph sampson breaks down iconic boston celt...,1378,230


### Copy dataframe - Data subset 1: `slct_tbl_full_df03`; data subset 2: `slct_tbl_full_df06`

Data subset 1 (`slct_tbl_full_df03`) will be used for removal of stop words and all punctuation, while data subset 2 (`slct_tbl_full_df06`) will retain stop words and selected EOS punctuation (i.e., periods).

In [65]:
slct_tbl_full_df06 = slct_tbl_full_df03.copy()

### Split text (tokenization 1: interim) - data subset 1a; column 1

#### Apply

Split text into tokens for addtional preprocessing.

In [66]:
slct_tbl_full_df03['article_split'] = \
slct_tbl_full_df03['article_no_punc_lower'].progress_apply(str.split)

100%|████| 36405/36405 [00:02<00:00, 13542.46it/s]


Print shape and first few records of resulting dataframe; also display a sample of full text.

In [67]:
ic(slct_tbl_full_df03.shape)
display(slct_tbl_full_df03.head())

ic| slct_tbl_full_df03.shape: (36405, 27)


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,...,author_source_trimmed,article_sep_emoji,article_urls_removed,author_source_urls_removed,article_w_one_return,article_punc_padded,article_no_punc,author_source_no_punc,article_no_punc_lower,article_split
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,"[have, you, ever, wanted, to, own, your, very,..."
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,"[over, five, months, after, julian, sands, wen..."
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,usa heels tar wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\nfour star running back picks michigan state ...,\nfour star running back picks michigan state ...,\nfour star running back picks michigan state ...,usa heels tar wire today,\nfour star running back picks michigan state ...,"[four, star, running, back, picks, michigan, s..."
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,usa roll tide wire today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\nalabama center charles bediako signs one yea...,\nalabama center charles bediako signs one yea...,\nalabama center charles bediako signs one yea...,usa roll tide wire today,\nalabama center charles bediako signs one yea...,"[alabama, center, charles, bediako, signs, one..."
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,usa wire celtics today,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\nralph sampson breaks down iconic boston celt...,\nralph sampson breaks down iconic boston celt...,\nralph sampson

In [68]:
for c in range(0,dply_rng_end02):
    try:
        print(slct_tbl_full_df03['article_split'][c],
              '\n')
    except:
        print(f'Skip {c}')

['have', 'you', 'ever', 'wanted', 'to', 'own', 'your', 'very', 'own', 'keg', 'of', 'titos', 'handmade', 'vodka', '.', 'now', 'that', 'dream', 'can', 'become', 'a', 'reality', '.', 'the', 'vodka', 'brand', 'is', 'launching', 'titos', 'in', 'a', 'big', 'can', 'a', 'ounce', 'stainless', 'steel', 'mini', 'cocktail', 'keg', '.', 'the', 'keg', 'includes', 'an', 'airtight', 'cap', 'a', 'co', 'regulator', 'stainless', 'steel', 'spear', 'tap', 'and', 'handle', 'and', 'an', 'extra', 'set', 'of', 'o', 'rings', '.', 'it', 'comes', 'empty', 'but', 'titos', 'said', 'it', 'can', 'be', 'filled', 'with', 'their', 'own', 'vodka', 'or', 'with', 'anything', 'else', '.', 'titos', 'is', 'also', 'releasing', 'titos', 'in', 'any', 'can', 'a', 'smaller', 'insulated', 'can', 'cooler', 'for', 'that', 'also', 'functions', 'as', 'a', 'refillable', 'cup', '.', 'it', 'can', 'fit', 'both', 'ounce', 'and', 'ounce', 'cans', 'and', 'converts', 'into', 'a', 'refillable', 'pint', 'tumbler', 'with', 'a', 'leak', 'proof', '

#### Display globally unique tokens

In [69]:
#print(uniq_tok(df_col=slct_tbl_full_df03['article_split']))

### Remove stop words - data subset 1a; column 1

Customize nltk list based on dataset specific values.

In [70]:
sw = stopwords.words("english")

# Customize nltk stop word list
sw.extend(['',
           '️',
           'arent',
           'cannot',
           'cant',
           'couldnt',
           'couldve',
           'didnt',
           'doesnt',
           'dont',
           'hadnt',
           'hasnt',
           'havent',
           'hes',
           'im',
           "i'm",
           'isnt',
           'it’s',
           'ive',
           '𝚘𝚏',
           'mightnt',
           'mustnt',
           'neednt',
           'shant',
           'shes',
           'shouldnt',
           'shouldve',
           'thatll',
           'theyll',
           'theyve',
           'wasnt',
           'werent',
           'whats',
           'weve',
           'wont',
           'wouldnt',
           'wouldve',
           'yall',
           'youd',
           'youll',
           'youre',
           'youve',
           "we'll",
           "you’re",
           "you’ve",
           "you’ll",
           "you’d",
           "she’s",
           "it’s",
           "that’ll",
           "don’t",
           "should’ve",
           "aren’t",
           "couldn’t",
           "didn’t",
           "doesn’t",
           "hadn’t",
           "hasn’t",
           "haven’t",
           "isn’t",
           "mightn’t",
           "mustn’t",
           "needn’t",
           "shan’t",
           "shouldn’t",
           "wasn’t",
           "weren’t",
           "won’t",
           "wouldn’t",
           "i’m",
           "we’ll",
           'said',
           'told',
           'according',
           'reporting',
           'reported',
           'statement',
           'spoke',
           'next',
           'though',
           'often',
           'story',
           'updated',
           'additional',
           'developments',
           'follow',
           'published',
           'com',
           'sunday',
           'monday',
           'tuesday',
           'wednesday',
           'thursday',
           'friday',
           'saturday',
           'january',
           'february',
           'march',
           'april',
           'may',
           'june',
           'july',
           'august',
           'september',
           'october',
           'november',
           'december',
           'via',
           'account',
           'accounts',
           'article',
           'advertisement',
           'advertisements',
          ])

print(len(sw))
print(sw)

292
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than

#### Create function

Define function to remove stop words in custom list.

In [71]:
def sw_remover(tokens):
    #tokens_p = [t for t in tokens if t.lower() not in sw]
    
    #return tokens_p
    return [t for t in tokens if t.lower() not in sw]

#### Call function

Apply custom function to entire text column of the dataframe.

In [72]:
slct_tbl_full_df03['article_near_final_split'] = \
slct_tbl_full_df03['article_split'].progress_apply(sw_remover)

100%|██████| 36405/36405 [01:13<00:00, 496.43it/s]


Print shape and first few records of resulting dataframe; also display a sample of full text.

In [73]:
ic(slct_tbl_full_df03.shape)
display(slct_tbl_full_df03.head())

ic| slct_tbl_full_df03.shape: (36405, 28)


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,...,article_sep_emoji,article_urls_removed,author_source_urls_removed,article_w_one_return,article_punc_padded,article_no_punc,author_source_no_punc,article_no_punc_lower,article_split,article_near_final_split
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,"[have, you, ever, wanted, to, own, your, very,...","[ever, wanted, keg, titos, handmade, vodka, .,..."
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,"[over, five, months, after, julian, sands, wen...","[five, months, julian, sands, went, missing, s..."
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\nfour star running back picks michigan state ...,\nfour star running back picks michigan state ...,\nfour star running back picks michigan state ...,usa heels tar wire today,\nfour star running back picks michigan state ...,"[four, star, running, back, picks, michigan, s...","[four, star, running, back, picks, michigan, s..."
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\nalabama center charles bediako signs one yea...,\nalabama center charles bediako signs one yea...,\nalabama center charles bediako signs one yea...,usa roll tide wire today,\nalabama center charles bediako signs one yea...,"[alabama, center, charles, bediako, signs, one...","[alabama, center, charles, bediako, signs, one..."
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\nralph sampson breaks d

In [74]:
for c in range(0,dply_rng_end02):
    try:
        print(slct_tbl_full_df03['article_near_final_split'][c],
              '\n')
    except:
        print(f'Skip {c}')

['ever', 'wanted', 'keg', 'titos', 'handmade', 'vodka', '.', 'dream', 'become', 'reality', '.', 'vodka', 'brand', 'launching', 'titos', 'big', 'ounce', 'stainless', 'steel', 'mini', 'cocktail', 'keg', '.', 'keg', 'includes', 'airtight', 'cap', 'co', 'regulator', 'stainless', 'steel', 'spear', 'tap', 'handle', 'extra', 'set', 'rings', '.', 'comes', 'empty', 'titos', 'filled', 'vodka', 'anything', 'else', '.', 'titos', 'also', 'releasing', 'titos', 'smaller', 'insulated', 'cooler', 'also', 'functions', 'refillable', 'cup', '.', 'fit', 'ounce', 'ounce', 'cans', 'converts', 'refillable', 'pint', 'tumbler', 'leak', 'proof', 'lid', '.', 'rebel', 'canformity', 'products', 'description', 'reads', '.', 'make', 'canned', 'cocktails', 'pre', 'mixes', 'seltzers', '.', '.', 'bestwhich', 'fast', 'food', 'restaurants', 'reign', 'supreme', '.', 'cast', 'votes', 'nowwhat', 'americans', 'drinking', 'summer', '.', 'tequila', 'modelo', 'canned', 'cocktails', 'non', 'alcoholic', 'drinksboth', 'products', '

#### Display globally unique tokens

In [75]:
#print(uniq_tok(df_col=slct_tbl_full_df03['article_near_final_split']))

### Split text (tokenization 2: interim) - data subsets 1b and 2; columns 1 and 2

#### Apply

Split text into tokens for addtional preprocessing.

In [76]:
slct_tbl_full_df03['author_source_no_punc_split'] = \
slct_tbl_full_df03['author_source_no_punc'].progress_apply(str.split)

'''_________________________________________________________________________'''

slct_tbl_full_df06['article_near_final_split'] = \
slct_tbl_full_df06['article_no_punc_lower'].progress_apply(str.split)

'''-------------------------------------------------------------------------'''

slct_tbl_full_df06['author_source_no_punc_split'] = \
slct_tbl_full_df06['author_source_no_punc'].progress_apply(str.split)

100%|████| 36405/36405 [00:01<00:00, 23826.98it/s]


Print shape and first few records of resulting dataframe; also display a sample of full text.

In [77]:
ic(slct_tbl_full_df03.shape)
display(slct_tbl_full_df03.head())

ic| slct_tbl_full_df03.shape: (36405, 29)


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,...,article_urls_removed,author_source_urls_removed,article_w_one_return,article_punc_padded,article_no_punc,author_source_no_punc,article_no_punc_lower,article_split,article_near_final_split,author_source_no_punc_split
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,"[have, you, ever, wanted, to, own, your, very,...","[ever, wanted, keg, titos, handmade, vodka, .,...","[usa, deletter, today, emily]"
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,"[over, five, months, after, julian, sands, wen...","[five, months, julian, sands, went, missing, s...","[usa, joy, today, ashford]"
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa heels tar wire today,\nfour star running back picks michigan state ...,\nfour star running back picks michigan state ...,\nfour star running back picks michigan state ...,usa heels tar wire today,\nfour star running back picks michigan state ...,"[four, star, running, back, picks, michigan, s...","[four, star, running, back, picks, michigan, s...","[usa, heels, tar, wire, today]"
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,usa roll tide wire today,\nalabama center charles bediako signs one yea...,\nalabama center charles bediako signs one yea...,\nalabama center charles bediako signs one yea...,usa roll tide wire today,\nalabama center charles bediako signs one yea...,"[alabama, center, charles, bediako, signs, one...","[alabama, center, charles, bediako, signs, one...","[usa, roll, tide, wire, today]"
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire celtics today,\nralph sampson breaks down iconic boston celt...,\nralph sampson breaks down iconic boston celt...,\nralph sampson breaks down iconic bo

In [78]:
for c in range(0,dply_rng_end02):
    try:
        print(slct_tbl_full_df03['article_near_final_split'][c],
              '\n')
    except:
        print(f'Skip {c}')

['ever', 'wanted', 'keg', 'titos', 'handmade', 'vodka', '.', 'dream', 'become', 'reality', '.', 'vodka', 'brand', 'launching', 'titos', 'big', 'ounce', 'stainless', 'steel', 'mini', 'cocktail', 'keg', '.', 'keg', 'includes', 'airtight', 'cap', 'co', 'regulator', 'stainless', 'steel', 'spear', 'tap', 'handle', 'extra', 'set', 'rings', '.', 'comes', 'empty', 'titos', 'filled', 'vodka', 'anything', 'else', '.', 'titos', 'also', 'releasing', 'titos', 'smaller', 'insulated', 'cooler', 'also', 'functions', 'refillable', 'cup', '.', 'fit', 'ounce', 'ounce', 'cans', 'converts', 'refillable', 'pint', 'tumbler', 'leak', 'proof', 'lid', '.', 'rebel', 'canformity', 'products', 'description', 'reads', '.', 'make', 'canned', 'cocktails', 'pre', 'mixes', 'seltzers', '.', '.', 'bestwhich', 'fast', 'food', 'restaurants', 'reign', 'supreme', '.', 'cast', 'votes', 'nowwhat', 'americans', 'drinking', 'summer', '.', 'tequila', 'modelo', 'canned', 'cocktails', 'non', 'alcoholic', 'drinksboth', 'products', '

#### Display globally unique tokens

In [79]:
#print(uniq_tok(df_col=slct_tbl_full_df03['article_near_final_split']))

#### Display globally unique tokens

In [80]:
#print(uniq_tok(df_col=slct_tbl_full_df03['author_source_no_punc_split']))

### Remove `author` and `source_name` values - data subsets 1 and 2

#### Create function

Define function to remove values from `author` and `source_name`, as their inclusion would add noise, given that they are source or article specific.

In [81]:
def auth_remover(tokens, other_column):
    #tokens_p = [t for t in tokens if t not in other_column]
    
    #return tokens_p
    return [t for t in tokens if t not in other_column]

#### Call function

Apply custom function to entire text column of the dataframe.

In [82]:
slct_tbl_full_df03['article_author_removed_split'] = slct_tbl_full_df03\
.progress_apply(lambda row: auth_remover(row['article_near_final_split'],
                                         row['author_source_no_punc_split']),
                axis=1)

'''_________________________________________________________________________'''

slct_tbl_full_df06['article_author_removed_split'] = slct_tbl_full_df06\
.progress_apply(lambda row: auth_remover(row['article_near_final_split'],
                                         row['author_source_no_punc_split']),
                axis=1)

100%|█████| 36405/36405 [00:03<00:00, 9632.68it/s]


Print shape and first few records of resulting dataframe; also display a sample of full text.

In [83]:
ic(slct_tbl_full_df03.shape)
display(slct_tbl_full_df03.head())

ic| slct_tbl_full_df03.shape: (36405, 30)


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,...,author_source_urls_removed,article_w_one_return,article_punc_padded,article_no_punc,author_source_no_punc,article_no_punc_lower,article_split,article_near_final_split,author_source_no_punc_split,article_author_removed_split
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,usa deletter today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,"[have, you, ever, wanted, to, own, your, very,...","[ever, wanted, keg, titos, handmade, vodka, .,...","[usa, deletter, today, emily]","[ever, wanted, keg, titos, handmade, vodka, .,..."
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,usa joy today ashford,over five months after julian sands went missi...,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,"[over, five, months, after, julian, sands, wen...","[five, months, julian, sands, went, missing, s...","[usa, joy, today, ashford]","[five, months, julian, sands, went, missing, s..."
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,usa heels tar wire today,\nfour star running back picks michigan state ...,\nfour star running back picks michigan state ...,\nfour star running back picks michigan state ...,usa heels tar wire today,\nfour star running back picks michigan state ...,"[four, star, running, back, picks, michigan, s...","[four, star, running, back, picks, michigan, s...","[usa, heels, tar, wire, today]","[four, star, running, back, picks, michigan, s..."
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,usa roll tide wire today,\nalabama center charles bediako signs one yea...,\nalabama center charles bediako signs one yea...,\nalabama center charles bediako signs one yea...,usa roll tide wire today,\nalabama center charles bediako signs one yea...,"[alabama, center, charles, bediako, signs, one...","[alabama, center, charles, bediako, signs, one...","[usa, roll, tide, wire, today]","[alabama, center, charles, bediako, signs, one..."
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,usa wire celtics today,\nralph sampson breaks down iconic boston celt...,\nralph sampson breaks down iconic boston celt...,\nralph sampson breaks down iconic boston celt...,usa wire celtics toda

In [84]:
for c in range(0,dply_rng_end02):
    try:
        print(slct_tbl_full_df03['article_author_removed_split'][c],
              '\n')
    except:
        print(f'Skip {c}')

['ever', 'wanted', 'keg', 'titos', 'handmade', 'vodka', '.', 'dream', 'become', 'reality', '.', 'vodka', 'brand', 'launching', 'titos', 'big', 'ounce', 'stainless', 'steel', 'mini', 'cocktail', 'keg', '.', 'keg', 'includes', 'airtight', 'cap', 'co', 'regulator', 'stainless', 'steel', 'spear', 'tap', 'handle', 'extra', 'set', 'rings', '.', 'comes', 'empty', 'titos', 'filled', 'vodka', 'anything', 'else', '.', 'titos', 'also', 'releasing', 'titos', 'smaller', 'insulated', 'cooler', 'also', 'functions', 'refillable', 'cup', '.', 'fit', 'ounce', 'ounce', 'cans', 'converts', 'refillable', 'pint', 'tumbler', 'leak', 'proof', 'lid', '.', 'rebel', 'canformity', 'products', 'description', 'reads', '.', 'make', 'canned', 'cocktails', 'pre', 'mixes', 'seltzers', '.', '.', 'bestwhich', 'fast', 'food', 'restaurants', 'reign', 'supreme', '.', 'cast', 'votes', 'nowwhat', 'americans', 'drinking', 'summer', '.', 'tequila', 'modelo', 'canned', 'cocktails', 'non', 'alcoholic', 'drinksboth', 'products', '

### Rejoin fully processed tokens - data subsets 1 and 2

#### Apply

Rejoin tokens for addtional preprocessing.

In [85]:
slct_tbl_full_df03['article_author_removed'] = \
slct_tbl_full_df03['article_author_removed_split'].progress_apply(" ".join)

'''_________________________________________________________________________'''

slct_tbl_full_df06['article_author_removed'] = \
slct_tbl_full_df06['article_author_removed_split'].progress_apply(" ".join)

100%|████| 36405/36405 [00:00<00:00, 52332.18it/s]


Display token count at current processing step.

In [86]:
token_cnt(df=slct_tbl_full_df03,
          df_col='article_author_removed')

100%|████| 36405/36405 [00:01<00:00, 35783.10it/s]

Total number of characters = 117668857
Total number of tokens = 17219371
Avg token len =  6.8335165668943425


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,...,article_no_punc,author_source_no_punc,article_no_punc_lower,article_split,article_near_final_split,author_source_no_punc_split,article_author_removed_split,article_author_removed,char_cnt,num_tokens
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,"[have, you, ever, wanted, to, own, your, very,...","[ever, wanted, keg, titos, handmade, vodka, .,...","[usa, deletter, today, emily]","[ever, wanted, keg, titos, handmade, vodka, .,...",ever wanted keg titos handmade vodka . dream b...,844,131
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,"[over, five, months, after, julian, sands, wen...","[five, months, julian, sands, went, missing, s...","[usa, joy, today, ashford]","[five, months, julian, sands, went, missing, s...",five months julian sands went missing solo hik...,1680,232
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,\nfour star running back picks michigan state ...,usa heels tar wire today,\nfour star running back picks michigan state ...,"[four, star, running, back, picks, michigan, s...","[four, star, running, back, picks, michigan, s...","[usa, heels, tar, wire, today]","[four, star, running, back, picks, michigan, s...",four star running back picks michigan state un...,1094,173
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,\nalabama center charles bediako signs one yea...,usa roll tide wire today,\nalabama center charles bediako signs one yea...,"[alabama, center, charles, bediako, signs, one...","[alabama, center, charles, bediako, signs, one...","[usa, roll, tide, wire, today]","[alabama, center, charles, bediako, signs, one...",alabama center charles bediako signs one year ...,1694,258
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,\nralph sampson breaks down iconic boston celt...,usa wire celtics today,\nralph sampson breaks down iconic boston celt...,"[ralph, sampson, breaks, down, iconic, boston,...","[ralph, sampson, breaks, iconic, boston, celti...","[usa, wire, celtics, today]","[ralph, sampson, breaks, iconic, boston, houst...",ralph sampson breaks iconic boston houston roc...,938,140


In [87]:
token_cnt(df=slct_tbl_full_df06,
          df_col='article_author_removed')

100%|████| 36405/36405 [00:01<00:00, 23937.82it/s]

Total number of characters = 164272022
Total number of tokens = 29049395
Avg token len =  5.654920593010629


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,...,article_punc_padded,article_no_punc,author_source_no_punc,article_no_punc_lower,article_near_final_split,author_source_no_punc_split,article_author_removed_split,article_author_removed,char_cnt,num_tokens
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,"[have, you, ever, wanted, to, own, your, very,...","[usa, deletter, today, emily]","[have, you, ever, wanted, to, own, your, very,...",have you ever wanted to own your very own keg ...,1199,224
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,over five months after julian sands went missi...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,"[over, five, months, after, julian, sands, wen...","[usa, joy, today, ashford]","[over, five, months, after, julian, sands, wen...",over five months after julian sands went missi...,2220,366
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,\nfour star running back picks michigan state ...,\nfour star running back picks michigan state ...,usa heels tar wire today,\nfour star running back picks michigan state ...,"[four, star, running, back, picks, michigan, s...","[usa, heels, tar, wire, today]","[four, star, running, back, picks, michigan, s...",four star running back picks michigan state ov...,1528,281
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,\nalabama center charles bediako signs one yea...,\nalabama center charles bediako signs one yea...,usa roll tide wire today,\nalabama center charles bediako signs one yea...,"[alabama, center, charles, bediako, signs, one...","[usa, roll, tide, wire, today]","[alabama, center, charles, bediako, signs, one...",alabama center charles bediako signs one year ...,2181,383
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,\nralph sampson breaks down iconic boston celt...,\nralph sampson breaks down iconic boston celt...,usa wire celtics today,\nralph sampson breaks down iconic boston celt...,"[ralph, sampson, breaks, down, iconic, boston,...","[usa, wire, celtics, today]","[ralph, sampson, breaks, down, iconic, boston,...",ralph sampson breaks down iconic boston housto...,1253,219


### Remove end-of-sentence (EOS) punctuation - data subsets 1 and 2

Create EOS punctuation set.

In [88]:
eos_punct = {"?", "!", "."}

#### Create function

Define function to remove EOS punctuation from tokens (but not processed full text).

In [89]:
def remove_eos(text,
               punct_set=eos_punct):
    #text_p = "".join([c for c in text if c not in punct_set])
    
    #return text_p
    return("".join([ch for ch in text if ch not in punct_set]))

#### Call function

Apply custom function to entire text column of the dataframe.

In [90]:
slct_tbl_full_df03['article_eos_removed'] = \
slct_tbl_full_df03['article_author_removed'].progress_apply(remove_eos,
                                                         punct_set=eos_punct)

'''_________________________________________________________________________'''

slct_tbl_full_df06['article_eos_removed'] = \
slct_tbl_full_df06['article_author_removed'].progress_apply(remove_eos,
                                                         punct_set=eos_punct)

100%|█████| 36405/36405 [00:08<00:00, 4134.76it/s]


Display token count at current processing step.

In [91]:
token_cnt(df=slct_tbl_full_df03,
          df_col='article_eos_removed')

100%|████| 36405/36405 [00:00<00:00, 37530.46it/s]

Total number of characters = 116239919
Total number of tokens = 15790433
Avg token len =  7.3614142816729595


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,...,author_source_no_punc,article_no_punc_lower,article_split,article_near_final_split,author_source_no_punc_split,article_author_removed_split,article_author_removed,article_eos_removed,char_cnt,num_tokens
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,usa deletter today emily,have you ever wanted to own your very own keg ...,"[have, you, ever, wanted, to, own, your, very,...","[ever, wanted, keg, titos, handmade, vodka, .,...","[usa, deletter, today, emily]","[ever, wanted, keg, titos, handmade, vodka, .,...",ever wanted keg titos handmade vodka . dream b...,ever wanted keg titos handmade vodka dream be...,830,117
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,usa joy today ashford,over five months after julian sands went missi...,"[over, five, months, after, julian, sands, wen...","[five, months, julian, sands, went, missing, s...","[usa, joy, today, ashford]","[five, months, julian, sands, went, missing, s...",five months julian sands went missing solo hik...,five months julian sands went missing solo hik...,1663,215
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,usa heels tar wire today,\nfour star running back picks michigan state ...,"[four, star, running, back, picks, michigan, s...","[four, star, running, back, picks, michigan, s...","[usa, heels, tar, wire, today]","[four, star, running, back, picks, michigan, s...",four star running back picks michigan state un...,four star running back picks michigan state un...,1082,161
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,usa roll tide wire today,\nalabama center charles bediako signs one yea...,"[alabama, center, charles, bediako, signs, one...","[alabama, center, charles, bediako, signs, one...","[usa, roll, tide, wire, today]","[alabama, center, charles, bediako, signs, one...",alabama center charles bediako signs one year ...,alabama center charles bediako signs one year ...,1673,237
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,usa wire celtics today,\nralph sampson breaks down iconic boston celt...,"[ralph, sampson, breaks, down, iconic, boston,...","[ralph, sampson, breaks, iconic, boston, celti...","[usa, wire, celtics, today]","[ralph, sampson, breaks, iconic, boston, houst...",ralph sampson breaks iconic boston houston roc...,ralph sampson breaks iconic boston houston roc...,932,134


In [92]:
token_cnt(df=slct_tbl_full_df06,
          df_col='article_eos_removed')

100%|████| 36405/36405 [00:01<00:00, 23463.93it/s]

Total number of characters = 162843084
Total number of tokens = 27620457
Avg token len =  5.895741840911612


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,...,article_no_punc,author_source_no_punc,article_no_punc_lower,article_near_final_split,author_source_no_punc_split,article_author_removed_split,article_author_removed,article_eos_removed,char_cnt,num_tokens
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,have you ever wanted to own your very own keg ...,usa deletter today emily,have you ever wanted to own your very own keg ...,"[have, you, ever, wanted, to, own, your, very,...","[usa, deletter, today, emily]","[have, you, ever, wanted, to, own, your, very,...",have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,1185,210
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,over five months after julian sands went missi...,usa joy today ashford,over five months after julian sands went missi...,"[over, five, months, after, julian, sands, wen...","[usa, joy, today, ashford]","[over, five, months, after, julian, sands, wen...",over five months after julian sands went missi...,over five months after julian sands went missi...,2203,349
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,\nfour star running back picks michigan state ...,usa heels tar wire today,\nfour star running back picks michigan state ...,"[four, star, running, back, picks, michigan, s...","[usa, heels, tar, wire, today]","[four, star, running, back, picks, michigan, s...",four star running back picks michigan state ov...,four star running back picks michigan state ov...,1516,269
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,\nalabama center charles bediako signs one yea...,usa roll tide wire today,\nalabama center charles bediako signs one yea...,"[alabama, center, charles, bediako, signs, one...","[usa, roll, tide, wire, today]","[alabama, center, charles, bediako, signs, one...",alabama center charles bediako signs one year ...,alabama center charles bediako signs one year ...,2160,362
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,\nralph sampson breaks down iconic boston celt...,usa wire celtics today,\nralph sampson breaks down iconic boston celt...,"[ralph, sampson, breaks, down, iconic, boston,...","[usa, wire, celtics, today]","[ralph, sampson, breaks, down, iconic, boston,...",ralph sampson breaks down iconic boston housto...,ralph sampson breaks down iconic boston housto...,1247,213


### Split text (tokenization 3: final) - data subsets 1 and 2

#### Apply

Split text into tokens for additonal EDA and machine learning methods.

In [93]:
slct_tbl_full_df03['processed_text_split'] = \
slct_tbl_full_df03['article_eos_removed'].progress_apply(str.split)

'''_________________________________________________________________________'''

slct_tbl_full_df06['processed_text_split'] = \
slct_tbl_full_df06['article_eos_removed'].progress_apply(str.split)

100%|█████| 36405/36405 [00:11<00:00, 3223.28it/s]


### Rejoin fully processed tokens - data subset 1

#### Apply

Rejoin tokens for additonal EDA and machine learning methods.

In [94]:
slct_tbl_full_df03['processed_text'] = \
slct_tbl_full_df03['processed_text_split'].progress_apply(" ".join)

100%|████| 36405/36405 [00:01<00:00, 23520.67it/s]


Display token count at current processing step.

In [95]:
token_cnt(df=slct_tbl_full_df03,
          df_col='processed_text')

100%|████| 36405/36405 [00:01<00:00, 24365.56it/s]


Total number of characters = 114810982
Total number of tokens = 15790433
Avg token len =  7.270920436444016


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,...,article_split,article_near_final_split,author_source_no_punc_split,article_author_removed_split,article_author_removed,article_eos_removed,processed_text_split,processed_text,char_cnt,num_tokens
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,"[have, you, ever, wanted, to, own, your, very,...","[ever, wanted, keg, titos, handmade, vodka, .,...","[usa, deletter, today, emily]","[ever, wanted, keg, titos, handmade, vodka, .,...",ever wanted keg titos handmade vodka . dream b...,ever wanted keg titos handmade vodka dream be...,"[ever, wanted, keg, titos, handmade, vodka, dr...",ever wanted keg titos handmade vodka dream bec...,816,117
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,"[over, five, months, after, julian, sands, wen...","[five, months, julian, sands, went, missing, s...","[usa, joy, today, ashford]","[five, months, julian, sands, went, missing, s...",five months julian sands went missing solo hik...,five months julian sands went missing solo hik...,"[five, months, julian, sands, went, missing, s...",five months julian sands went missing solo hik...,1646,215
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,"[four, star, running, back, picks, michigan, s...","[four, star, running, back, picks, michigan, s...","[usa, heels, tar, wire, today]","[four, star, running, back, picks, michigan, s...",four star running back picks michigan state un...,four star running back picks michigan state un...,"[four, star, running, back, picks, michigan, s...",four star running back picks michigan state un...,1070,161
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,"[alabama, center, charles, bediako, signs, one...","[alabama, center, charles, bediako, signs, one...","[usa, roll, tide, wire, today]","[alabama, center, charles, bediako, signs, one...",alabama center charles bediako signs one year ...,alabama center charles bediako signs one year ...,"[alabama, center, charles, bediako, signs, one...",alabama center charles bediako signs one year ...,1652,237
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,"[ralph, sampson, breaks, down, iconic, boston,...","[ralph, sampson, breaks, iconic, boston, celti...","[usa, wire, celtics, today]","[ralph, sampson, breaks, iconic, boston, houst...",ralph sampson breaks iconic boston houston roc...,ralph sampson breaks iconic boston houston roc...,"[ralph, sampson, breaks, iconic, boston, houst...",ralph sampson breaks iconic boston ho

### Collapse EOS periods for dataset full text with stop words - data subset 2

#### Create function

Define function to use regular expression to eliminate space before period.

In [96]:
rex_eospc = rex.compile(r'\s\.')

def collapse_period(text):
    repl_text = rex_eospc.sub(r'.', text)
    
    return repl_text

#### Call function

Apply custom function to entire text column of the dataframe.

In [97]:
slct_tbl_full_df06['processed_text'] = \
slct_tbl_full_df06['article_author_removed'].progress_apply(collapse_period)

100%|████| 36405/36405 [00:01<00:00, 19052.07it/s]


Display token count at current processing step.

In [98]:
token_cnt(df=slct_tbl_full_df06,
          df_col='processed_text')

100%|████| 36405/36405 [00:02<00:00, 15790.44it/s]

Total number of characters = 162843100
Total number of tokens = 27620473
Avg token len =  5.89573900490408


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,...,article_no_punc_lower,article_near_final_split,author_source_no_punc_split,article_author_removed_split,article_author_removed,article_eos_removed,processed_text_split,processed_text,char_cnt,num_tokens
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,have you ever wanted to own your very own keg ...,"[have, you, ever, wanted, to, own, your, very,...","[usa, deletter, today, emily]","[have, you, ever, wanted, to, own, your, very,...",have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"[have, you, ever, wanted, to, own, your, very,...",have you ever wanted to own your very own keg ...,1185,210
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,over five months after julian sands went missi...,"[over, five, months, after, julian, sands, wen...","[usa, joy, today, ashford]","[over, five, months, after, julian, sands, wen...",over five months after julian sands went missi...,over five months after julian sands went missi...,"[over, five, months, after, julian, sands, wen...",over five months after julian sands went missi...,2203,349
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,\nfour star running back picks michigan state ...,"[four, star, running, back, picks, michigan, s...","[usa, heels, tar, wire, today]","[four, star, running, back, picks, michigan, s...",four star running back picks michigan state ov...,four star running back picks michigan state ov...,"[four, star, running, back, picks, michigan, s...",four star running back picks michigan state ov...,1516,269
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,\nalabama center charles bediako signs one yea...,"[alabama, center, charles, bediako, signs, one...","[usa, roll, tide, wire, today]","[alabama, center, charles, bediako, signs, one...",alabama center charles bediako signs one year ...,alabama center charles bediako signs one year ...,"[alabama, center, charles, bediako, signs, one...",alabama center charles bediako signs one year ...,2160,362
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,\nralph sampson breaks down iconic boston celt...,"[ralph, sampson, breaks, down, iconic, boston,...","[usa, wire, celtics, today]","[ralph, sampson, breaks, down, iconic, boston,...",ralph sampson breaks down iconic boston housto...,ralph sampson breaks down iconic boston housto...,"[ralph, sampson, breaks, down, iconic, boston,...",ralph sampson breaks down iconic bost

### Count characters and words for initial review - data subsets 1 and 2

Calculate number of tokens and number of characters.

In [99]:
# Raw text character and word counts
slct_tbl_full_df03['char_cnt'] = slct_tbl_full_df03['processed_text']\
.progress_apply(len)
slct_tbl_full_df03['num_tokens'] = slct_tbl_full_df03['processed_text']\
.progress_apply(lambda x: len(x.split()))
display(slct_tbl_full_df03.head())

'''_________________________________________________________________________'''

slct_tbl_full_df06['char_cnt'] = slct_tbl_full_df06['processed_text']\
.progress_apply(len)
slct_tbl_full_df06['num_tokens'] = slct_tbl_full_df06['processed_text']\
.progress_apply(lambda x: len(x.split()))

display(slct_tbl_full_df06.head())

100%|████| 36405/36405 [00:01<00:00, 24966.87it/s]


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,...,article_split,article_near_final_split,author_source_no_punc_split,article_author_removed_split,article_author_removed,article_eos_removed,processed_text_split,processed_text,char_cnt,num_tokens
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,"[have, you, ever, wanted, to, own, your, very,...","[ever, wanted, keg, titos, handmade, vodka, .,...","[usa, deletter, today, emily]","[ever, wanted, keg, titos, handmade, vodka, .,...",ever wanted keg titos handmade vodka . dream b...,ever wanted keg titos handmade vodka dream be...,"[ever, wanted, keg, titos, handmade, vodka, dr...",ever wanted keg titos handmade vodka dream bec...,816,117
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,"[over, five, months, after, julian, sands, wen...","[five, months, julian, sands, went, missing, s...","[usa, joy, today, ashford]","[five, months, julian, sands, went, missing, s...",five months julian sands went missing solo hik...,five months julian sands went missing solo hik...,"[five, months, julian, sands, went, missing, s...",five months julian sands went missing solo hik...,1646,215
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,"[four, star, running, back, picks, michigan, s...","[four, star, running, back, picks, michigan, s...","[usa, heels, tar, wire, today]","[four, star, running, back, picks, michigan, s...",four star running back picks michigan state un...,four star running back picks michigan state un...,"[four, star, running, back, picks, michigan, s...",four star running back picks michigan state un...,1070,161
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,"[alabama, center, charles, bediako, signs, one...","[alabama, center, charles, bediako, signs, one...","[usa, roll, tide, wire, today]","[alabama, center, charles, bediako, signs, one...",alabama center charles bediako signs one year ...,alabama center charles bediako signs one year ...,"[alabama, center, charles, bediako, signs, one...",alabama center charles bediako signs one year ...,1652,237
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,"[ralph, sampson, breaks, down, iconic, boston,...","[ralph, sampson, breaks, iconic, boston, celti...","[usa, wire, celtics, today]","[ralph, sampson, breaks, iconic, boston, houst...",ralph sampson breaks iconic boston houston roc...,ralph sampson breaks iconic boston houston roc...,"[ralph, sampson, breaks, iconic, boston, houst...",ralph sampson breaks iconic boston ho

100%|████| 36405/36405 [00:02<00:00, 15432.13it/s]


,text_id,source_name,author,title,url,publish_date,article_text,content,article_lower,author_source,...,article_no_punc_lower,article_near_final_split,author_source_no_punc_split,article_author_removed_split,article_author_removed,article_eos_removed,processed_text_split,processed_text,char_cnt,num_tokens
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,have you ever wanted to own your very own keg ...,"[have, you, ever, wanted, to, own, your, very,...","[usa, deletter, today, emily]","[have, you, ever, wanted, to, own, your, very,...",have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"[have, you, ever, wanted, to, own, your, very,...",have you ever wanted to own your very own keg ...,1185,210
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,over five months after julian sands went missi...,"[over, five, months, after, julian, sands, wen...","[usa, joy, today, ashford]","[over, five, months, after, julian, sands, wen...",over five months after julian sands went missi...,over five months after julian sands went missi...,"[over, five, months, after, julian, sands, wen...",over five months after julian sands went missi...,2203,349
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,\nfour star running back picks michigan state ...,"[four, star, running, back, picks, michigan, s...","[usa, heels, tar, wire, today]","[four, star, running, back, picks, michigan, s...",four star running back picks michigan state ov...,four star running back picks michigan state ov...,"[four, star, running, back, picks, michigan, s...",four star running back picks michigan state ov...,1516,269
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,\nalabama center charles bediako signs one yea...,"[alabama, center, charles, bediako, signs, one...","[usa, roll, tide, wire, today]","[alabama, center, charles, bediako, signs, one...",alabama center charles bediako signs one year ...,alabama center charles bediako signs one year ...,"[alabama, center, charles, bediako, signs, one...",alabama center charles bediako signs one year ...,2160,362
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,\nralph sampson breaks down iconic boston celt...,"[ralph, sampson, breaks, down, iconic, boston,...","[usa, wire, celtics, today]","[ralph, sampson, breaks, down, iconic, boston,...",ralph sampson breaks down iconic boston housto...,ralph sampson breaks down iconic boston housto...,"[ralph, sampson, breaks, down, iconic, boston,...",ralph sampson breaks down iconic bost

Display a sample of full text - data subset 1

In [100]:
display_example(df=slct_tbl_full_df03,
                df_iter_col='source_name',
                txt_col='processed_text',
                iter_lst=source_name_lst,
                upper_rng=dply_rng_end02
               )

ic| 1699879310.py:20 in display_example() at 10:00:18.630
ic| 1699879310.py:37 in display_example() at 10:00:18.719
ic| 1699879310.py:37 in display_example() at 10:00:18.761
ic| 1699879310.py:20 in display_example() at 10:00:18.804


6752
2
[6532, 2501]
'USA Today' article #1
URL: https://ftw.usatoday.com/article/how-to-watch-david-horsey-at-the-u-s-open-live-stream-tv-channel-odds
Author(s): Data Skrive
Title: How to Watch David Horsey at the U.S. Open: Live Stream, TV Channel, Odds
watch david horsey us open live stream tv channel odds share share tweet text email link facebook twitter text message linkedin email watch david horsey u open live stream tv channel odds pm et us open underway david horsey th position score watch david horsey us open date tv network location los angeles california course los angeles country club live stream fubo watch free espn new home pga tour live sign access hours live coverage pga tour tournaments year david horseys last five tournaments date tournament finish score earnings genesis scottish open mc recent stats horsey finished last four rounds bogey free finished last four rounds top score day horsey finished within five strokes best score day last four rounds david horsey odds 

ic| 1699879310.py:37 in display_example() at 10:00:18.882
ic| 1699879310.py:37 in display_example() at 10:00:18.927
ic| 1699879310.py:20 in display_example() at 10:00:18.981
ic| 1699879310.py:37 in display_example() at 10:00:19.042
ic| 1699879310.py:

'ABC News' article #1
URL: https://abcnews.go.com/US/wireStory/paris-summit-aims-shake-financial-system-test-leaders-100265400
Author(s): FATIMA HUSSEIN and PAUL WISEMAN Associated Press
Title: Paris summit aims to shake up the financial system. It will test leaders' resolve on climate
paris summit aims shake financial system test leaders resolve climateheads state finance leaders activists converging paris seek ways overhaul international monetary fund world bank help weather warmer stormier worldbyfatima pressjune amclimate activists stage demonstration transform eiffel tower wind turbine ahead global climate finance summit paris heads state finance leaders activists around world converge paris week summit aimed discussing overhaul worlds development banks like world bank imf usher post pandemic world rapidly changing climate ap photomichel eulerthe presswashington heads state finance leaders activists around world converge paris week seek ways overhaul worlds development banks like 

37 in display_example() at 10:00:19.081
ic| 1699879310.py:20 in display_example() at 10:00:19.127
ic| 1699879310.py:37 in display_example() at 10:00:19.180
ic| 1699879310.py:37 in display_example() at 10:00:19.231
ic| 1699879310.py:

...

7786
2
[7024, 908]
'Forbes' article #1
URL: https://www.forbes.com/sites/victorlopez-carmen/2023/06/04/indigenous-health-at-stake-supreme-courts-indian-child-welfare-act-verdict-looms/
Author(s): Victor A. Lopez-Carmen, Contributor, 
 Victor A. Lopez-Carmen, Contributor
 https://www.forbes.com/sites/victorlopez-carmen/
Title: Indigenous Health At Stake: Supreme Court’s Indian Child Welfare Act Verdict Looms
comes championing health indigenous children united states pieces legislation possess profound significance indian child welfare act icwa merely dry legal document lifeline cultural health indigenous nations countless children foster care system coming weeks supreme court united states scotus expected release decision haaland vs brackeen case challenges constitutionality icwa plaintiffs non indigenous texan couple named chad jennifer brackeen claim icwas preference placing indigenous children foster parents tribal nations discriminates non indigenous individuals wish adopt indigenou

) at 10:00:21.080
ic| 1699879310.py:37 in display_example() at 10:00:21.121
ic| 1699879310.py:20 in display_example() at 10:00:21.171
ic| 1699879310.py:37 in display_example() at 10:00:21.231


'PEOPLE' article #1
URL: https://people.com/jessica-simpson-photo-of-daughter-birdie-and-pet-dog-7510727
Author(s): People
Title: Jessica Simpson Shares Sweet Photo of Daughter Birdie, 4, and Their Pet Dog: ‘My Sugar Cookies’
celebrity parents jessica simpson shares sweet photo daughter birdie pet dog sugar cookies mom three also parent ace knute maxwell maxi drew escher walcott edt jessica simpson daughter birdie photo jessica simpson instagram jessica simpson enjoying time sweet family mom three shared adorable photo instagram youngest daughter birdie mae cuddling familys pet pooch dixie spent quality time cute pic birdie simpson shares husband eric johnson flashed big smile wrapped arms around fluffy dog sweet outfit included pink dress multicolored tutu skirt accessorized pink green beaded necklace hair styled pigtails matching flower hair clips simpson called girls sugar cookies caption photo seen second photo posted beaming beside daughter dixie joined family appeared try sneak s

ic| 1699879310.py:37 in display_example() at 10:00:21.274
ic| 1699879310.py:20 in display_example() at 10:00:21.319
ic| 1699879310.py:37 in display_example() at 10:00:21.375
ic| 1699879310.py:37 in display_example() at 10:00:21.422


'CNBC' article #2
URL: https://www.cnbc.com/2023/05/31/salesforce-ceo-marc-benioff-says-new-ai-enhanced-products-will-be-a-revelation.html
Author(s): Julie Coleman
Title: Salesforce CEO Marc Benioff says new AI-enhanced products will be a 'revelation'
watch nowvideo salesforce ceo marc benioff says ai enabled slack become copilotmad money jim cramersalesforce ceo marc benioff cnbcs jim cramer artificial intelligence already boon cloud software company especially comes one recently acquired products messaging application slack dream data thats inside slack well work large language model slack almost going like wake become intelligent benioff cramer going incredible revelation customers got intelligence side slack slack announced earlier month plans add whole host generative ai features program including slack gpt summarize messages take notes even help improve message tone among things means slack going assistant partner copilot helping run company salesforce gpt going let copilot runni

Display a sample of full text - data subset 2

In [101]:
display_example(df=slct_tbl_full_df06,
                df_iter_col='source_name',
                txt_col='processed_text',
                iter_lst=source_name_lst,
                upper_rng=dply_rng_end02
               )

ic| 1699879310.py:20 in display_example() at 10:00:21.622
ic| 1699879310.py:37 in display_example() at 10:00:21.685
ic| 1699879310.py:37 in display_example() at 10:00:21.729
ic| 1699879310.py:20 in display_example() at 10:00:21.760
ic| 1699879310.py:37 in display_example() at 10:00:21.830
ic

6752
2
[5776, 4557]
'USA Today' article #1
URL: https://steelerswire.usatoday.com/2023/06/10/pittsburgh-steelers-diontae-johnson/
Author(s): Steelers Wire
Title: Check out Steelers WR Diontae Johnson working hard in the offseason
check out wr diontae johnson working hard in the offseason share this article shares share tweet text email link on facebook on twitter via text message on linkedin via email diontae johnson curt popejoy june pm et last season no wide receiver was better at getting open in the nfl than pittsburgh steeler diontae johnson. judging by the video below that johnson shared on instagram that isnt going to change anytime soon. johnson shared a video of him grinding on the field working hard on his quickness and cuts showing those lightning fast moves and zero wasted energy. johnson will forever be known as the guy the drafted with the pick acquired from the trade that sent antonio brown to the las vegas raiders. hes picked up the mantle as the teams no. wide receiver 

| 1699879310.py:37 in display_example() at 10:00:21.868
ic| 1699879310.py:20 in display_example() at 10:00:21.904
ic| 1699879310.py:37 in display_example() at 10:00:21.963
ic| 1699879310.py:37 in display_example() at 10:00:22.007
ic| 1699879310.py:20 in display_example() at 10:00:22.046


'ABC News' article #2
URL: https://abcnews.go.com/International/wireStory/3-year-probe-encrypted-phones-led-seizure-hundreds-100403271
Author(s): MIKE CORDER Associated Press
Title: 3-year probe into encrypted phones led to seizure of tons of drugs, prosecutors say
a year probe of encrypted phones led to the seizure of hundreds of tons of drugs prosecutors sayinternational prosecutors say that investigations triggered by the cracking of encrypted phones three years ago have so far led to more than arrests worldwide and the seizure of hundreds of tons of drugsbymike pressjune ambaudoin thouvenot french national member at eurojust european union agency for criminal justice cooperation left and alexander van dam national member for netherlands at eurojust attend a conference in lille northern france tuesday june. the announcement tuesday underscores the staggering scale of criminality mainly drugs and arms smuggling and money laundering that was uncovered as a result of police and prosecu

ic| 1699879310.py:37 in display_example() at 10:00:22.106
ic| 1699879310.py:37 in display_example() at 10:00:22.153
ic| 1699879310.py:20 in display_example() at 10:00:22.191
ic| 1699879310.

...

'Forbes' article #2
URL: https://www.forbes.com/sites/forbestechcouncil/2023/06/28/intelligent-transformation-in-a-multipolar-world-why-organizational-resilience-is-critical/
Author(s): Mike Ettling, Forbes Councils Member, 
 Mike Ettling, Forbes Councils Member
 https://www.forbes.com/sites/forbestechcouncil/people/mikeettling1/
Title: Intelligent Transformation In A Multipolar World: Why Organizational Resilience Is Critical
ceo of unit transforming the people experience of enterprise software. getty it is time for organizations both large and small to start thinking the unthinkable. what might have been considered unimaginable before the financial crash in particularly among the established old world western economies is very rapidly crystalizing into what could become the largest reshaping of global power and economic structures since the second world war. as a south african native i have always had a slightly different perspective on globalization and free market economics so i am

00:23.790
ic| 1699879310.py:37 in display_example() at 10:00:23.845
ic| 1699879310.py:37 in display_example() at 10:00:23.885
ic| 1699879310.py:20 in display_example() at 10:00:23.924
ic| 1699879310.py:37 in display_example() at 10:00:23.968
ic| 1699879310.py:

1338
2
[696, 839]
'CNBC' article #1
URL: https://www.cnbc.com/2023/06/13/google-rto-crackdown-gets-backlash-check-my-work-not-my-badge.html
Author(s): Jennifer Elias
Title: Google's return-to-office crackdown gets backlash from some employees: 'Check my work, not my badge'
in this articlegooglgoogfollow your favorite stockscreate free account google ceo sundar pichai speaks during the google io keynote session at shoreline amphitheatre in mountain view california may. josh edelson afp googles mixed messaging when it comes to its return to office plans has been a subject of consternation across the company since the waning days of the covid pandemic. now employees are finding further sources of frustration. last week google updated its hybrid three day a week office policy to include badge tracking and noted attendance will be included in performance reviews. additionally employees who already received approval for remote work may now have that status reevaluated. based on cnbcs discuss

37 in display_example() at 10:00:24.006


'Vox' article #2
URL: https://www.vox.com/culture/23762279/the-idol-hbo-review-sex-scenes-lily-rose-depp-boring
Author(s): Alex Abad-Santos
Title: The “edgy” sex at the center of HBO’s The Idol is the least interesting part
filed under culture tv hbos the idol should just be funny the edgy sex at the center of the idol is the least interesting part of the show. by jun pm edt share this story share this on facebook share this on twitter share all sharing options share all sharing options for hbos the idol should just be funny reddit pocket flipboard email lily rose depp in hbos the idol. eddy chenhbo is a senior correspondent who explains what society obsesses over from marvel and movies to fitness and skin care. he came to in. prior to that he worked at the atlantic. there was a time before porn was widely available on the internet when horny teenage boys had to be resourceful. stealing playboys and penthouses from older siblings ruining their eyesight watching scrambled discolored wav

Set list of dataframe columns to export.

In [102]:
export_col_names_lst = ['text_id',
                        'source_name',
                        'author',
                        'title',
                        'url',
                        'publish_date',
                        'article_text',
                        'content',
                        'processed_text',
                        'processed_text_split',
                        'num_tokens',
                       ]

### Write file without stop words to CSV - data subset 1

Set path to write CSV file to.

In [103]:
file_out_name01 = f'data_preprocessed_wo_sw_{today}.csv'
file_out_path01 = os.path.join(up1_dir, data_location, file_out_name01)
print(f'CSV file out 1 path: {file_out_path01}')

CSV file out 1 path: C:\Users\acarr\Documents\GitHub\599_team_project\data\data_preprocessed_wo_sw_2023-07-23_09-55-15672808.csv


Write pandas dataframe to CSV; save locally.

In [104]:
if to_csv_flag == False:
    pass
else:
    slct_tbl_full_df03[export_col_names_lst].to_csv(file_out_path01,
                                                    index=False)

In [105]:
print(slct_tbl_full_df03.shape)

(36405, 36)


#### Display globally unique tokens

In [106]:
df03_t = uniq_tok(df_col=slct_tbl_full_df03['processed_text_split'])

222591

Elapsed processing time = 3.4478116999999884


In [107]:
ic(len(df03_t))
print(slct_tbl_full_df03['num_tokens'].sum())
#print(df03_t)

ic| len(df03_t): 222591


15790433


### Write file with stop words to CSV - data subset 2

Set path to write CSV file to.

In [108]:
file_out_name02 = f'data_preprocessed_w_sw_{today}.csv'
file_out_path02 = os.path.join(up1_dir, data_location, file_out_name02)
print(f'CSV file out 2 path: {file_out_path02}')

CSV file out 2 path: C:\Users\acarr\Documents\GitHub\599_team_project\data\data_preprocessed_w_sw_2023-07-23_09-55-15672808.csv


Write pandas dataframe to CSV; save locally.

In [109]:
if to_csv_flag == False:
    pass
else:
    slct_tbl_full_df06[export_col_names_lst].to_csv(file_out_path02,
                                                    index=False)

In [110]:
print(slct_tbl_full_df06.shape)

(36405, 35)


#### Display globally unique tokens

In [111]:
df06_t = uniq_tok(df_col=slct_tbl_full_df06['processed_text_split'])

222812

Elapsed processing time = 4.953895299999999


In [112]:
ic(len(df06_t))
print(slct_tbl_full_df06['num_tokens'].sum())
#print(df06_t)

ic| len(df06_t): 222812


27620473


### Keyword-in-Context (KWIC)

Define function to display KWIC (Albrecht et al., 2021).

In [113]:
def kwic(doc_series, keyword, window=35, print_samples=5):
    '''Search article text for keywords in context (KWIC)'''
    def add_kwic(text):
        kwic_list.extend(keyword_in_context(doc=text,
                                            keyword=keyword,
                                            ignore_case=True,
                                            window_width=window))
    kwic_list = []
    doc_series.map(add_kwic)
    
    if print_samples is None or print_samples==0:
        return kwic_list
    else:
        k = min(print_samples, len(kwic_list))
        print(f"{k} random samples out of {len(kwic_list)}" + \
              f" contexts for '{keyword}':")
        for sample in random.sample(list(kwic_list), k):
            ic()
            display(sample[0] + sample[1] + sample[2])
            print('\n')

#### Call function

In [114]:
#kwic(slct_tbl_full_df03['article_text'], 'amp', window=150, print_samples=120)

kwic(slct_tbl_full_df03['article_rex_trimmed'],
     "amazon bites",
     window=200,
     print_samples=500)

0 random samples out of 0 contexts for 'amazon bites':


### Sandbox for testing application of complex regular expressions to clean data

Example of a function used for identifying instances of regular expressions.

In [115]:
#rex_find_comp01 = rex.compile(r'[\n]+')
#rex_find_comp01 = rex.compile(r'like this article\?')
#rex_find_comp01 = rex.compile(r'like this article\?[\n]+')
#rex_find_comp01 = rex.compile(r'like this article\?[\n]+sign[(up)|(for)|(the)|\s]+\s')
#rex_find_comp01 = rex.compile(r'like this article\?[\n]+sign[(up)|(for)|(the)|\s]+\s[\w\s]*newsletter to get our top stories in your inbox every morning[\n,\s,\t]')
'''_________________________________________________________________________'''
#rex_find_comp01 = rex.compile(r'click(?:here|to|get|ge|the|for|sign|read|up|our|hear|more|instagram|twitter|view|learn|\s)*[\w\s]*(?:newsletter|app|reporting|cartoons){1}\b')
'''_________________________________________________________________________'''
#rex_find_comp01 = rex.compile(r'copyright\s[\d]+[\w\s]*[^\n]+[\.|\n]{1}')
'''_________________________________________________________________________'''
#rex_find_comp01 = rex.compile(r'email[\n|\t|\s]+sign up[\n|\t|\s]+view all newsletters[\n|\t|\s]+more[\w\s]+[\n|\t|\s]{1}')
'''_________________________________________________________________________'''
#rex_find_comp01 = rex.compile(r'gannett[\S|\s|]*gamble\.')
'''_________________________________________________________________________'''
#rex_find_comp01 = rex.compile(r'\n\ncatch the excitement and bet with tipico!\ndouble down, ohio! deposit $200, get $250. bet now! in colorado and new jersey, claim your 100% deposit match up to $250 now! 21+, see tipico.com for terms and conditions.')
#rex_find_comp01 = rex.compile(r'catch the excitement[\S|\s|\n]+conditions.')
'''_________________________________________________________________________'''
#rex_find_comp01 = rex.compile(r'must be\s[\d]+[\S|\s]+apply\.')
'''_________________________________________________________________________'''
#rex_find_comp01 = rex.compile(r'[ ]{2,}')
'''_________________________________________________________________________'''
#rex_find_comp01 = rex.compile(r'[\.]{3}')
'''_________________________________________________________________________'''
#rex_find_comp01 = rex.compile(r'\b\d+\.\d+\b')
#rex_find_comp01 = rex.compile(r'(?:[a-z]\.){2,}')
'''_________________________________________________________________________'''
#rex_find_comp01 = rex.compile(r'\b[\s]*more[\s]*from[\s]forbes')
#rex_find_comp01 = rex.compile(r'\b[\s]*moreby[\w|\s]*(?:press|staff)+')
#rex_find_comp01 = rex.compile(r'\b[\s]*tap to play gif[\s]*\b')
'''_________________________________________________________________________'''
#rex_find_comp01 = rex.compile(r'jason[\s]*abbruzzese')
#rex_find_comp01 = rex.compile(r'amazon[s]*(?:ian)*')
#rex_find_comp01 = rex.compile(r'(?:un)*account(?:ing|ability|able|ant|ancy|ed|ably)*(?:s)?')
#rex_find_comp01 = rex.compile(r'advertisement(?:s)*')
#rex_find_comp01 = rex.compile(r'(?:p)*article(?:s)*')
#rex_find_comp01 = rex.compile(r'contributorfacebookpinteresttwittermaillink')
#rex_find_comp01 = rex.compile(r'(?:un)*account(?:ing|ability|able|ant|ancy|ed|ably)*(?:s)?')
#rex_find_comp01 = rex.compile(r'advertisement(?:s)*')
#rex_find_comp01 = rex.compile(r'share to(?:facebook|share|to|twitter|linkedin|\s)+')
#rex_find_comp01 = rex.compile(r'by\s*amy\s*danise\s*editor')
#rex_find_comp01 = rex.compile(r'explainerinformationwired')
#rex_find_comp01 = rex.compile(r'\b(?:photo|by|\s)*[\w\s]*getty[\s]*images[\s]*')
'''_________________________________________________________________________'''
#rex_find_comp01 = rex.compile(r'\.[\w|\s]*amazon[\w|\s]*\.')

#rex_find_comp01 = rex.compile(r'\.[\w|\s]*amazon[\s*]check[\w|\s]*\.')
#rex_find_comp01 = rex.compile(r'\.\n+\s*amazon[\w|\s]*\n+\s*\.')
#rex_find_comp01 = rex.compile(r'\.[\s]*amazon[\w\s]*small(?:biz|business|\s)+[\w\s]*\.')
#rex_find_comp01 = rex.compile(r'\.\n+\s*shop\s*amazon[\w|\s]*(?:\s\n)+')
#rex_find_comp01 = rex.compile(r'\.\s*(?:buy|sign up|you can)+[\w\s]*amazon[\w|\s]*\.')
'''_________________________________________________________________________'''
# Not used
#rex_find_comp01 = rex.compile(r'\b[\w]*article[\w]*\b')
'''_________________________________________________________________________'''
#rex_find_comp01 = rex.compile(r'[\s]*getty[\s]*images[\s]*')
#rex_find_comp01 = rex.compile(r'(note to user[\S\s]+license agreement\b){1}')
rex_find_comp01 = rex.compile(r'\b[\w]*by[\w]*\b')
rex_find_comp01 = rex.compile(r'\b[\w]*link[\w]*\b')
rex_find_comp01 = rex.compile(r'click(?:here|to|get|ge|the|for|sign|read|up|our|hear|more|instagram|twitter|view|learn|\s)*[\w\s]*(?:newsletter|app|reporting|cartoons){1}\b')



rnt_lst01 = []

def find_rex_in_txt(text):
    if rex_find_comp01.findall(text) == []:
        pass
    else:
        rnt_lst01.append(rex_find_comp01.findall(text))
    
#slct_tbl_full_df03['article_w_one_return'].progress_apply(find_rex_in_txt)
slct_tbl_full_df03['article_norm'].progress_apply(find_rex_in_txt)

print(len(rnt_lst01[:]))
rnt_flat_lst01 = list(itertools.chain.from_iterable(rnt_lst01))
rnt_flat_set01 = set(rnt_flat_lst01)
rnt_flat_lst01 = list(rnt_flat_set01)
print(len(sorted(rnt_flat_lst01)))
display(sorted(rnt_flat_lst01[:]))
#display(sorted(rnt_flat_lst01[-1000:]))

100%|████| 36405/36405 [00:01<00:00, 18986.18it/s]

1357
18


['click here click here to get the fox news app',
 'click here for more cartoons',
 'click here for the fox news app',
 'click here to download the fox news app',
 'click here to get on the newsletter',
 'click here to get the fox news app',
 'click here to get the opinion newsletter',
 'click here to sign up for our entertainment newsletter',
 'click here to sign up for our health newsletter',
 'click here to sign up for our lifestyle newsletter',
 'click here to sign up for our opinion newsletter',
 'click here to sign up for the entertainment newsletter',
 'click here to sign up for the fox news autos newsletter',
 'click here to sign up for the future of work newsletter',
 'click here to subscribe to our newsletter',
 'click on the survey link to secure your spot as one of the select few beta testers of the fanatics sportsbook app',
 'click the button above to start betting on the app',
 'click to get the fox news app']

### Calculate concentration ratio of each set of corpora

Define function to calculate the concentration ratios of words in one source's articles relative to the others. The output potentially provides insights into words that appear in one source but are used very infrequently (if at all) in other sources' articles.

In [116]:
# Capstone note: Extended custom function to use bi- and tri-grams
def concen_ratio(header_lst=[],
                 token_lsts=[],
                 ng_start=1,
                 ng_end=1):
    '''
    Accept 2-item (1) title and (2) token lists;
    output concentration ratios for one vs. other
    '''
    lyr_corp_lst = []
    for l in token_lsts:
        #print(type(l))
        lyr_corp_lst.append(' '.join(l))
    #print(len(lyr_corp_lst))
    #print(lyr_corp_lst)

    cv = CountVectorizer(input='content',
                         encoding='utf-8',
                         stop_words=None,
                         token_pattern=r'\S+',
                         ngram_range=(ng_start, ng_end)
                        )

    lyr_tokens_fit = cv.fit(lyr_corp_lst)
    
    try:
        print('Random sample of n-grams')
        print(pd.Series(cv.get_feature_names()).sample(3))
    except:
        pass

    lyr_tokens_sm = cv.transform(lyr_corp_lst)
    display(lyr_tokens_sm)

    df = pd.DataFrame(lyr_tokens_sm.toarray(),
                      columns=cv.get_feature_names())
    #display(df)

    df02 = df.copy()
    df02['r_sum'] = df02.sum(axis=1)
    #display(df02)

    '''Filter by frequency for all columns citation:
    1. https://chat.openai.com/share/38cdf818-6d2a-4ebb-b6c2-edb3a99a6d02
    2. https://pandas.pydata.org/pandas-docs/stable/reference/api/
    pandas.DataFrame.ge.html
    '''
    condition = df.ge(5).all()
    #print(condition)

    # Get the list of columns that satisfy the condition
    columns = condition[condition].index.tolist()
    #print(columns)
    columns.append('r_sum')
    #print(columns)

    #display(df02[columns])

    df03 = df02[columns].copy()
    #display(df03)

    '''Row proportions citation:
    https://chat.openai.com/share/f429fbb6-9bdc-47e8-bee9-697df28a6fac
    '''
    df04 = df03.progress_apply(lambda x: x / df03.iloc[:,-1],
                               axis=0)
    #display(df04)

    # Create new rows by dividing one artist row by the second artists row
    new_row01 = df04.iloc[0] / df04.iloc[1]
    new_row02 = df04.iloc[1] / df04.iloc[0]

    # Append the new row to the DataFrame
    df04 = df04.append(new_row01,
                       ignore_index=True)
    df04 = df04.append(new_row02,
                       ignore_index=True)
    #display(df04)

    # Transpose dataframe
    df05 = df04.T
    df05 = df05.reset_index()
    df05.columns = ['token',
                    'c1_concen',
                    'c2_concen',
                    'c1c2_concen_ratio',
                    'c2c1_concen_ratio']
    #print(df05)
    
    '''Sort values citation:
    https://pandas.pydata.org/pandas-docs/stable/reference/api
    /pandas.DataFrame.sort_values.html
    '''
    #print(header_lst[0])
    df05a = df05[['token',
                  'c1c2_concen_ratio']].sort_values(by='c1c2_concen_ratio',
                                                    ascending=False).head(15)
    #print(header_lst[1])
    df05b = df05[['token',
                  'c2c1_concen_ratio']].sort_values(by='c2c1_concen_ratio',
                                                    ascending=False).head(15)
    
    '''Display dataframes side by side citation:
    https://softhints.com
    /display-two-pandas-dataframes-side-by-side-jupyter-notebook/
    '''    
    df1_styler = df05a.style.set_table_attributes("style='display:inline'")\
    .set_caption(header_lst[0])
    df2_styler = df05b.style.set_table_attributes("style='display:inline'")\
    .set_caption(header_lst[1])

    display_html(df1_styler._repr_html_() + df2_styler._repr_html_(),
                 raw=True)

    
def run_cr(df=pd.DataFrame(),
           ref_lst=[],
           token_col=None,
           ref_col=None):
    '''
    Create 1-vs.-all dataframes;
    run concen_ratio iteratively to display comparison
    for each source
    '''
    start_time = time.perf_counter()
    df01 = df.copy()
    df01['token_split'] = df01[token_col].progress_apply(str.split)
    for i in ref_lst:
        ic()
        df01a = df01[df01[ref_col] == i]
        print(f"Shape of '{i}' dataframe: {df01a.shape}")
        #display(df01a.head(2))
        df01a_s1 = list(itertools.chain\
                        .from_iterable(list(pd.Series(df01a['token_split']))))
        #print(df01a_s1[:10])

        df01b = df01[df01[ref_col] != i]
        print(f"Shape of not-'{i}' dataframe: {df01b.shape}")
        #display(df01b.head(2))
        df01b_s1 = list(itertools.chain\
                        .from_iterable(list(pd.Series(df01b['token_split']))))
        #print(df01b_s1[:10])
        for n in range(4,7):
            ic()
            concen_ratio(header_lst=[f'1({i})-to-all: {n} tokens',
                                     f'all-to-1({i}): {n} tokens'],
                         token_lsts=[df01a_s1, df01b_s1],
                         ng_start=n,
                         ng_end=n)
    end_time = time.perf_counter()
    print(f'\nElapsed processing time = {end_time - start_time}')

#### Call function

Apply custom function to entire text column of the dataframe.

### Pipeline consolidation

#### Pipeline function

In [117]:
def prepare(text, pipeline):
    '''Run a pipeline of text processing transformers'''
    tokens = str(text)
    
    # Pull key and val from trans dictionaries
    for transformer in pipeline:
        trans = list(transformer.keys())[0]
        args = list(transformer.values())[0]
        #print(trans)
        #print(args)
        if args == None:
            #print(1)
            tokens = trans(tokens)
        else:
            #print('check99', trans, args)
            tokens = trans(tokens, args) 

    return(tokens)

#### `article_text` preprocessing - w/o lemmatization

## Display runtime

In [118]:
global_end_time = time.perf_counter()

In [119]:
print(f'''\nElapsed processing time = {round((global_end_time 
- global_start_time)/60,2)} mins''')


Elapsed processing time = 5.42 mins


## References

* Albrecht, J., Ramachandran, S., & Winkler, C. (2021). *Blueprints for text analytics using Python: Machine learning-based solutions for common real world (NLP) applications*. O’Reilly.